 test 4
 multi strategy
 non working: matplotlib results page, and report generator. 
 

In [ ]:
#!/usr/bin/env python
# coding: utf-8

# Wolf's AlgoHaus Backtester v7.0 - Enhanced Multi-Strategy Edition
# Wolf Guzman
# Features: Parallel Strategy Execution, Editable Strategies, Checkbox Selection
# Professional Forex Backtesting with Parquet Data

import customtkinter as ctk
import tkinter as tk
from tkinter import messagebox, filedialog, scrolledtext
import pandas as pd
import numpy as np
from datetime import datetime, timedelta, date
import webbrowser
import os
import sys
import tempfile
import inspect
import pathlib
import threading
import queue
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import logging
import matplotlib
matplotlib.use('Agg')  # Use non-interactive backend
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from matplotlib.figure import Figure
import seaborn as sns
import json
import base64
from io import BytesIO
from concurrent.futures import ProcessPoolExecutor, ThreadPoolExecutor, as_completed
import multiprocessing as mp

ctk.set_appearance_mode("dark")
ctk.set_default_color_theme("green")
logging.basicConfig(level=logging.INFO, format='%(levelname)s: %(message)s')


# ======================================================================
# 1. FOREX CALCULATOR (UNCHANGED - OPTIMIZED)
# ======================================================================
class ForexCalculator:
    """Handle all forex calculations - Wolf Guzman's Trading System"""

    LEVERAGE_OPTIONS = [1, 10, 20, 30, 50, 100, 200, 500]

    PIP_VALUES = {
        # Major Pairs
        'EUR/USD': 0.0001, 'GBP/USD': 0.0001, 'USD/JPY': 0.01,
        'USD/CHF': 0.0001, 'USD/CAD': 0.0001, 'AUD/USD': 0.0001,
        'NZD/USD': 0.0001,
        # Cross Pairs - JPY
        'EUR/JPY': 0.01, 'GBP/JPY': 0.01, 'AUD/JPY': 0.01,
        'NZD/JPY': 0.01, 'CHF/JPY': 0.01, 'CAD/JPY': 0.01,
        # Cross Pairs - EUR
        'EUR/GBP': 0.0001, 'EUR/CHF': 0.0001, 'EUR/AUD': 0.0001,
        # Cross Pairs - GBP
        'GBP/CHF': 0.0001,
        # Cross Pairs - AUD
        'AUD/CHF': 0.0001, 'AUD/NZD': 0.0001,
        # Cross Pairs - NZD
        'NZD/CHF': 0.0001,
        # Cross Pairs - CAD
        'CAD/CHF': 0.0001,
        # Exotic
        'USD/THB': 0.01
    }

    # Hardcoded date ranges from Wolf's data for faster loading
    DATA_RANGES = {
        'AUD/CHF': ('2014-12-09', '2025-10-06'),
        'AUD/JPY': ('2014-12-10', '2025-10-06'),
        'AUD/NZD': ('2014-12-10', '2025-10-06'),
        'AUD/USD': ('2014-12-09', '2025-10-06'),
        'CAD/CHF': ('2014-12-10', '2025-10-06'),
        'CHF/JPY': ('2014-12-08', '2025-10-06'),
        'EUR/AUD': ('2014-12-09', '2025-10-06'),
        'EUR/CHF': ('2014-12-10', '2025-10-06'),
        'EUR/GBP': ('2014-12-08', '2025-10-06'),
        'EUR/JPY': ('2014-12-10', '2025-10-06'),
        'EUR/USD': ('2014-12-09', '2025-10-06'),
        'GBP/CHF': ('2014-12-10', '2025-10-06'),
        'GBP/USD': ('2014-12-10', '2025-10-06'),
        'NZD/CHF': ('2014-12-09', '2025-10-06'),
        'NZD/JPY': ('2014-12-08', '2025-10-06'),
        'NZD/USD': ('2014-12-10', '2025-10-06'),
        'USD/CAD': ('2014-12-10', '2025-10-06'),
        'USD/CHF': ('2014-12-09', '2025-10-06'),
        'USD/JPY': ('2014-12-09', '2025-10-06'),
        'USD/THB': ('2014-12-03', '2025-10-06'),
    }

    USD_MAJORS = {
        'EUR/USD', 'GBP/USD', 'AUD/USD', 'NZD/USD', 
        'USD/JPY', 'USD/CHF', 'USD/CAD', 'USD/THB'
    }

    @staticmethod
    def is_usd_major(pair):
        return pair in ForexCalculator.USD_MAJORS

    @staticmethod
    def calculate_pip_value_in_usd(pair, unit_size, current_price, conversion_rate=1.0):
        """Calculate pip value in USD for given position size"""
        pip_size = ForexCalculator.PIP_VALUES.get(pair, 0.0001) 

        if pair.endswith('/USD'):
            pip_value = pip_size * unit_size
        elif pair.startswith('USD/'):
            pip_value = (pip_size / current_price) * unit_size
        else:
            pip_value_in_base = pip_size * unit_size
            pip_value = pip_value_in_base * conversion_rate 

        return pip_value

    @staticmethod
    def calculate_margin_required(pair, unit_size, current_price, leverage, conversion_rate=1.0):
        """Calculate margin required for position"""
        if pair.endswith('/USD'):
            position_value = unit_size * current_price
        elif pair.startswith('USD/'):
            position_value = unit_size
        else:
            position_value = unit_size * conversion_rate 

        margin_required = position_value / leverage
        return margin_required

    @staticmethod
    def calculate_position_size(balance, risk_pct, sl_pips, pair, price, conversion_rate=1.0):
        """Calculate position size based on risk percentage"""
        risk_amount = balance * (risk_pct / 100)
        pip_val_per_unit = ForexCalculator.calculate_pip_value_in_usd(pair, 1, price, conversion_rate)
        if pip_val_per_unit <= 0:
            return 0
        size = risk_amount / (sl_pips * pip_val_per_unit)
        return max(1000, int(round(size / 1000)) * 1000)


# ======================================================================
# 2. DATA LOADING WITH VALIDATION (OPTIMIZED WITH CACHING)
# ======================================================================

# Global cache for loaded data
_DATA_CACHE = {}
_CACHE_LOCK = threading.Lock()

def detect_available_pairs(base_folder: pathlib.Path):
    """Scan for available forex pair folders and return valid pairs - OPTIMIZED"""
    pairs = set()

    if not base_folder.exists():
        logging.error(f"Base folder does not exist: {base_folder}")
        return []

    logging.info(f"Scanning for pairs in: {base_folder}")

    # Use list comprehension for faster iteration
    subfolders = [f for f in base_folder.iterdir() if f.is_dir() and f.name not in ['README.TXT', '__pycache__']]
    
    for subfolder in subfolders:
        folder_name = subfolder.name
        if '_' in folder_name and len(folder_name.split('_')) == 2:
            parts = folder_name.split('_')
            if len(parts[0]) == 3 and len(parts[1]) == 3:
                # Check for parquet files without listing all
                if any(subfolder.glob("*.parquet")):
                    pair = folder_name.replace('_', '/')
                    pairs.add(pair)
                    logging.info(f"Found pair: {pair}")

    if not pairs:
        logging.warning("No valid pairs found!")
    else:
        logging.info(f"Total pairs found: {len(pairs)}")

    return sorted(list(pairs))

def get_data_date_range(pair_name: str, base_folder: pathlib.Path):
    """Get actual date range from parquet file - OPTIMIZED with cache"""
    if pair_name in ForexCalculator.DATA_RANGES:
        start_str, end_str = ForexCalculator.DATA_RANGES[pair_name]
        start = datetime.strptime(start_str, '%Y-%m-%d').date()
        end = datetime.strptime(end_str, '%Y-%m-%d').date()
        return start, end

    try:
        pair_folder_name = pair_name.replace('/', '_')
        pair_folder = base_folder / pair_folder_name

        if not pair_folder.exists() or not pair_folder.is_dir():
            logging.warning(f"Folder not found: {pair_folder}")
            return None, None

        parquet_files = list(pair_folder.glob("*.parquet"))

        if not parquet_files:
            logging.warning(f"No parquet files in {pair_folder}")
            return None, None

        # Read only metadata for date range (much faster)
        df = pd.read_parquet(parquet_files[0], engine='pyarrow', columns=None)
        
        datetime_col = None
        for col in df.columns:
            if any(keyword in col.lower() for keyword in ['datetime', 'date', 'time']):
                datetime_col = col
                break

        if datetime_col:
            df[datetime_col] = pd.to_datetime(df[datetime_col], errors='coerce', utc=True)
            df = df.dropna(subset=[datetime_col])
            df[datetime_col] = df[datetime_col].dt.tz_localize(None)
            start = df[datetime_col].min().date()
            end = df[datetime_col].max().date()
            return start, end

        return None, None
    except Exception as e:
        logging.error(f"Error getting date range for {pair_name}: {e}")
        return None, None

def load_pair_data(pair_name: str, base_folder: pathlib.Path, start_date: datetime, end_date: datetime, timeframe: str):
    """Load and validate parquet data - OPTIMIZED with caching and vectorization"""
    
    # Create cache key
    cache_key = f"{pair_name}_{start_date.date()}_{end_date.date()}_{timeframe}"
    
    # Check cache first
    with _CACHE_LOCK:
        if cache_key in _DATA_CACHE:
            logging.info(f"Using cached data for {pair_name}")
            cached_df, actual_start, actual_end = _DATA_CACHE[cache_key]
            return cached_df.copy(), actual_start, actual_end
    
    pair_folder_name = pair_name.replace('/', '_')
    pair_folder = base_folder / pair_folder_name

    logging.info(f"=" * 50)
    logging.info(f"Loading pair: {pair_name}")

    if not pair_folder.exists():
        raise FileNotFoundError(f"Folder does not exist: {pair_folder}")

    parquet_files = list(pair_folder.glob("*.parquet"))

    if not parquet_files:
        raise FileNotFoundError(f"No PARQUET files found in {pair_folder}")

    parquet_path = parquet_files[0]

    # Load with pyarrow for speed
    df = pd.read_parquet(parquet_path, engine='pyarrow')

    if df.empty:
        raise ValueError(f"PARQUET file is empty: {parquet_path}")

    # Optimized column mapping
    cols_lower = {c.strip().lower(): c for c in df.columns}
    col_map = {
        'datetime': next((cols_lower[k] for k in ['datetime', 'date', 'time', 'timestamp'] if k in cols_lower), None),
        'open': next((cols_lower[k] for k in ['open', 'o'] if k in cols_lower), None),
        'high': next((cols_lower[k] for k in ['high', 'h'] if k in cols_lower), None),
        'low': next((cols_lower[k] for k in ['low', 'l'] if k in cols_lower), None),
        'close': next((cols_lower[k] for k in ['close', 'c', 'last'] if k in cols_lower), None),
        'volume': next((cols_lower[k] for k in ['volume', 'vol', 'v'] if k in cols_lower), None)
    }

    # Rename columns
    rename_dict = {v: k for k, v in col_map.items() if v is not None}
    df = df.rename(columns=rename_dict)

    # Add volume if missing
    if 'volume' not in df.columns:
        df['volume'] = 1000

    # Vectorized datetime conversion
    df['datetime'] = pd.to_datetime(df['datetime'], errors='coerce', utc=True)
    df = df.dropna(subset=['datetime'])
    df['datetime'] = df['datetime'].dt.tz_localize(None)

    # Remove duplicates and sort
    df = df.drop_duplicates(subset='datetime', keep='first')
    df = df.sort_values('datetime')

    actual_start = df['datetime'].min().date()
    actual_end = df['datetime'].max().date()

    # Adjust dates if needed
    if start_date.date() < actual_start:
        start_date = datetime.combine(actual_start, datetime.min.time())

    if end_date.date() > actual_end:
        end_date = datetime.combine(actual_end, datetime.min.time())

    # Filter data
    df = df.set_index('datetime')
    user_start = max(pd.Timestamp(start_date.date()), df.index.min())
    user_end = min(pd.Timestamp(end_date.date()) + pd.Timedelta(hours=23, minutes=59, seconds=59), df.index.max())
    df = df.loc[user_start:user_end].copy()

    if df.empty:
        raise ValueError(f"No data in range {start_date.date()} to {end_date.date()}")

    # Resample if needed
    if timeframe != '1min':
        rule = {'5min': '5T', '15min': '15T', '1hr': '1H', '1Day': '1D'}.get(timeframe, '1T')
        df = df.resample(rule).agg({
            'open': 'first', 'high': 'max', 'low': 'min', 'close': 'last', 'volume': 'sum'
        }).dropna()

    df = df.reset_index()
    df['date'] = df['datetime'].dt.date

    # Vectorized daily aggregations
    daily = df.groupby('date').agg({
        'high': 'max', 'low': 'min', 'close': 'last'
    })
    daily.columns = ['day_high', 'day_low', 'day_close']
    daily['prev_high'] = daily['day_high'].shift(1)
    daily['prev_low'] = daily['day_low'].shift(1)
    daily['prev_close'] = daily['day_close'].shift(1)

    daily = daily.reset_index()
    df = pd.merge(df, daily[['date', 'prev_high', 'prev_low', 'prev_close']], on='date', how='left')
    df[['prev_high', 'prev_low', 'prev_close']] = df[['prev_high', 'prev_low', 'prev_close']].ffill()

    logging.info(f"Loaded {len(df)} bars")
    
    # Cache the result
    with _CACHE_LOCK:
        _DATA_CACHE[cache_key] = (df.copy(), actual_start, actual_end)
    
    return df, actual_start, actual_end


# ======================================================================
# 3. EDITABLE TRADING STRATEGIES
# ======================================================================
class TradingStrategies:
    """Editable trading strategies with improved logic"""
    
    @staticmethod
    def vwap_crossover_strategy(df, sl_pips, tp_pips, pip_value):
        """VWAP Crossover Strategy - OPTIMIZED"""
        df = df.copy()
        
        # Vectorized VWAP calculation
        df['tpv'] = df['volume'] * (df['high'] + df['low'] + df['close']) / 3
        df['cumvol'] = df.groupby('date')['volume'].cumsum()
        df['cumtpv'] = df.groupby('date')['tpv'].cumsum()
        df['vwap'] = df['cumtpv'] / df['cumvol']

        # Vectorized signal generation
        df['prev_close'] = df['close'].shift(1)
        df['prev_vwap'] = df['vwap'].shift(1)

        df['signal'] = None
        buy_condition = (df['prev_close'] <= df['prev_vwap']) & (df['close'] > df['vwap'])
        sell_condition = (df['prev_close'] >= df['prev_vwap']) & (df['close'] < df['vwap'])
        df.loc[buy_condition, 'signal'] = 'BUY'
        df.loc[sell_condition, 'signal'] = 'SELL'

        # Generate trades
        entries = df[df['signal'].notna()].copy()
        trades = []

        for idx, row in entries.iterrows():
            remaining_data = df[df.index > idx].reset_index(drop=True)
            if len(remaining_data) > 0:
                trades.append({
                    'datetime': row['datetime'],
                    'entry_price': row['close'],
                    'signal': row['signal'],
                    'day_data': remaining_data
                })

        return trades

    @staticmethod
    def opening_range_strategy(df, sl_pips, tp_pips, pip_value):
        """Opening Range Breakout Strategy - FIXED AND OPTIMIZED"""
        df = df.copy()
        trades = []

        # Group by date for efficiency
        for date in df['date'].unique():
            day_data = df[df['date'] == date].reset_index(drop=True)
            
            # Need at least 31 bars (30 for OR + 1 for breakout)
            if len(day_data) < 31: 
                continue

            # Opening Range: first 30 minutes (bars)
            opening_range = day_data.iloc[:30]
            or_high = opening_range['high'].max()
            or_low = opening_range['low'].min()

            # FIXED: Check for valid range
            if or_high == or_low:
                continue  # Skip if no range formed
            
            breakout_detected = False
            
            # Start checking from bar 30 onwards
            for i in range(30, len(day_data)):
                if breakout_detected:
                    break

                bar = day_data.iloc[i]

                # FIXED: Use high/low for breakout detection, close for entry
                # BUY: High breaks above OR high
                if bar['high'] > or_high:
                    trades.append({
                        'datetime': bar['datetime'],
                        'entry_price': or_high,  # Enter at breakout level
                        'signal': 'BUY',
                        'day_data': day_data[i+1:].reset_index(drop=True)
                    })
                    breakout_detected = True

                # SELL: Low breaks below OR low
                elif bar['low'] < or_low:
                    trades.append({
                        'datetime': bar['datetime'],
                        'entry_price': or_low,  # Enter at breakout level
                        'signal': 'SELL',
                        'day_data': day_data[i+1:].reset_index(drop=True)
                    })
                    breakout_detected = True

        return trades

    @staticmethod
    def bollinger_band_reversion_strategy(df, sl_pips, tp_pips, pip_value, period=20, std_dev=2):
        """Bollinger Band Mean Reversion Strategy - OPTIMIZED"""
        df = df.copy()

        # Vectorized BB calculation
        df['bb_middle'] = df['close'].rolling(window=period).mean()
        df['bb_std'] = df['close'].rolling(window=period).std()
        df['bb_upper'] = df['bb_middle'] + (std_dev * df['bb_std'])
        df['bb_lower'] = df['bb_middle'] - (std_dev * df['bb_std'])

        # Vectorized signal generation
        df['signal'] = None
        buy_condition = df['close'] < df['bb_lower']
        sell_condition = df['close'] > df['bb_upper']
        df.loc[buy_condition, 'signal'] = 'BUY'
        df.loc[sell_condition, 'signal'] = 'SELL'

        entries = df[df['signal'].notna()].copy()
        trades = []

        for idx, row in entries.iterrows():
            remaining_data = df[df.index > idx].reset_index(drop=True)
            if len(remaining_data) > 0:
                trades.append({
                    'datetime': row['datetime'],
                    'entry_price': row['close'],
                    'signal': row['signal'],
                    'day_data': remaining_data,
                    'bb_upper': row['bb_upper'],
                    'bb_middle': row['bb_middle'],
                    'bb_lower': row['bb_lower']
                })

        return trades


# ======================================================================
# 4. ENHANCED BACKTESTER WITH PARALLEL PROCESSING
# ======================================================================
class EnhancedBacktester:
    def __init__(self, df, initial_balance=10000, pip_value=0.0001, leverage=50, 
                 risk_percent=1.0, spread_pips=1.5, slippage_pips=0.5):
        self.df = df
        self.initial_balance = initial_balance
        self.pip_value = pip_value
        self.leverage = leverage
        self.risk_percent = risk_percent
        self.spread_pips = spread_pips
        self.slippage_pips = slippage_pips
        self.results = None

    def run_backtest(self, strategy_func, sl_pips, tp_pips, pair_name, progress_callback=None):
        """Run backtest with vectorized operations for speed"""
        logging.info(f"Generating trades using {strategy_func.__name__}...")

        if progress_callback:
            progress_callback(0, "Generating trade signals...")

        trades = strategy_func(self.df, sl_pips, tp_pips, self.pip_value)

        if not trades:
            self.results = pd.DataFrame()
            return "No trades generated.", {}

        logging.info(f"Processing {len(trades)} potential trades...")
        results = []
        current_balance = self.initial_balance
        trade_number = 1

        is_usd_major = ForexCalculator.is_usd_major(pair_name)

        total_trades = len(trades)

        for idx, t in enumerate(trades):
            if progress_callback and (idx % max(1, total_trades // 100) == 0 or idx == total_trades - 1):
                progress = int((idx / total_trades) * 100)
                progress_callback(progress, f"Processing trade {idx+1}/{total_trades} ({progress}%)")

            entry_price = t['entry_price']
            signal = t['signal']
            bars = t['day_data']

            if bars.empty:
                continue

            unit_size = ForexCalculator.calculate_position_size(
                current_balance, self.risk_percent, sl_pips, pair_name, entry_price
            )

            if unit_size < 1000:
                continue

            margin_required = ForexCalculator.calculate_margin_required(
                pair_name, unit_size, entry_price, self.leverage, 1.0
            )

            if margin_required > current_balance * 0.8:
                continue

            pip_value_usd = ForexCalculator.calculate_pip_value_in_usd(
                pair_name, unit_size, entry_price, 1.0
            )

            spread_cost = self.spread_pips * self.pip_value
            slippage_cost = self.slippage_pips * self.pip_value

            if signal == 'BUY':
                actual_entry_price = entry_price + spread_cost + slippage_cost
            else:
                actual_entry_price = entry_price - spread_cost - slippage_cost

            if signal == 'BUY':
                stop_level = actual_entry_price - (sl_pips * self.pip_value)
                take_level = actual_entry_price + (tp_pips * self.pip_value)
            else:
                stop_level = actual_entry_price + (sl_pips * self.pip_value)
                take_level = actual_entry_price - (tp_pips * self.pip_value)

            # Vectorized exit detection
            exit_idx = None
            exit_reason = 'Timeout'
            exit_price = None

            if signal == 'BUY':
                sl_hit = bars['low'] <= stop_level
                tp_hit = bars['high'] >= take_level
            else:
                sl_hit = bars['high'] >= stop_level
                tp_hit = bars['low'] <= take_level

            # Find first exit
            sl_indices = sl_hit[sl_hit].index
            tp_indices = tp_hit[tp_hit].index

            if len(sl_indices) > 0 and len(tp_indices) > 0:
                if sl_indices[0] < tp_indices[0]:
                    exit_idx = sl_indices[0]
                    exit_price = stop_level
                    exit_reason = 'SL'
                else:
                    exit_idx = tp_indices[0]
                    exit_price = take_level
                    exit_reason = 'TP'
            elif len(sl_indices) > 0:
                exit_idx = sl_indices[0]
                exit_price = stop_level
                exit_reason = 'SL'
            elif len(tp_indices) > 0:
                exit_idx = tp_indices[0]
                exit_price = take_level
                exit_reason = 'TP'

            if exit_price is None:
                exit_idx = len(bars) - 1
                exit_price = bars.iloc[-1]['close']
                exit_reason = 'Timeout'

            if signal == 'BUY':
                actual_exit_price = exit_price - slippage_cost
            else:
                actual_exit_price = exit_price + slippage_cost

            if signal == 'BUY':
                pips_pnl = (actual_exit_price - actual_entry_price) / self.pip_value
            else:
                pips_pnl = (actual_entry_price - actual_exit_price) / self.pip_value

            monetary_pnl = pips_pnl * pip_value_usd

            entry_time = t['datetime']
            exit_time = bars.iloc[exit_idx]['datetime'] if exit_idx is not None else bars.iloc[-1]['datetime']

            time_in_trade = exit_time - entry_time
            hours_in_trade = time_in_trade.total_seconds() / 3600
            current_balance += monetary_pnl

            results.append({
                'trade_number': f"{trade_number:05d}",
                'entry_time': entry_time,
                'exit_time': exit_time,
                'time_in_trade_hours': round(hours_in_trade, 2),
                'signal': signal,
                'entry_price': round(actual_entry_price, 5),
                'exit_price': round(actual_exit_price, 5),
                'exit_reason': exit_reason,
                'pips_pnl': round(pips_pnl, 2),
                'pips': round(pips_pnl, 2),  # Add 'pips' column for CSV export
                'monetary_pnl': round(monetary_pnl, 2),
                'spread_cost_usd': round(self.spread_pips * pip_value_usd, 2),
                'slippage_cost_usd': round(self.slippage_pips * pip_value_usd * 2, 2),
                'unit_size': unit_size,
                'margin_used': round(margin_required, 2),
                'balance': round(current_balance, 2),
                'pip_value_usd': round(pip_value_usd, 4)
            })

            trade_number += 1

        if progress_callback:
            progress_callback(100, "Calculating metrics...")

        logging.info(f"Backtest complete: {len(results)} trades executed")
        self.results = pd.DataFrame(results)

        if not self.results.empty:
            total_pnl = self.results['monetary_pnl'].sum()
            total_pips = self.results['pips_pnl'].sum()
            win_rate = (self.results['pips_pnl'] > 0).mean() * 100
            total_return_pct = ((current_balance - self.initial_balance) / self.initial_balance) * 100

            summary = (f"TRADES: {len(self.results)}\n"
                      f"WIN RATE: {win_rate:.1f}%\n"
                      f"P&L: ${total_pnl:,.2f}\n"
                      f"PIPS: {total_pips:,.1f}\n"
                      f"RETURNS: {total_return_pct:+.2f}%\n"
                      f"FINAL BALANCE: ${current_balance:,.2f}")

            if not is_usd_major:
                summary += "\n⚠️ Cross pair - approximate pip values"

            metrics = self.calculate_metrics()
            metrics['total_return_pct'] = total_return_pct
        else:
            summary = "No trades executed"
            metrics = {}

        return summary, metrics

    def calculate_metrics(self):
        """Calculate comprehensive trading metrics - OPTIMIZED"""
        trades_df = self.results
        if trades_df.empty:
            return {}

        total_trades = len(trades_df)
        winning_trades = (trades_df['monetary_pnl'] > 0).sum()
        losing_trades = (trades_df['monetary_pnl'] < 0).sum()
        win_rate = (winning_trades / total_trades * 100) if total_trades > 0 else 0

        total_pnl = trades_df['monetary_pnl'].sum()
        total_pips = trades_df['pips_pnl'].sum()

        wins = trades_df[trades_df['monetary_pnl'] > 0]['monetary_pnl']
        losses = trades_df[trades_df['monetary_pnl'] < 0]['monetary_pnl']

        avg_win = wins.mean() if len(wins) > 0 else 0
        avg_loss = abs(losses.mean()) if len(losses) > 0 else 0
        largest_win = wins.max() if len(wins) > 0 else 0
        largest_loss = abs(losses.min()) if len(losses) > 0 else 0

        gross_profit = wins.sum() if len(wins) > 0 else 0
        gross_loss = abs(losses.sum()) if len(losses) > 0 else 0
        profit_factor = gross_profit / gross_loss if gross_loss > 0 else float('inf')

        final_balance = trades_df['balance'].iloc[-1]
        total_return = ((final_balance - self.initial_balance) / self.initial_balance) * 100

        # Vectorized drawdown calculation
        equity_curve = self.initial_balance + trades_df['monetary_pnl'].cumsum()
        cummax = equity_curve.expanding().max()
        drawdown = (equity_curve - cummax) / cummax * 100
        max_drawdown_pct = drawdown.min()

        if len(trades_df) > 1:
            daily_returns = trades_df.groupby(trades_df['entry_time'].dt.date)['monetary_pnl'].sum()
            daily_returns_pct = daily_returns / self.initial_balance

            sharpe = (daily_returns_pct.mean() * 252) / (daily_returns_pct.std() * np.sqrt(252)) if daily_returns_pct.std() > 0 else 0

            negative_returns = daily_returns_pct[daily_returns_pct < 0]
            downside_std = negative_returns.std() if len(negative_returns) > 0 else 0
            sortino = (daily_returns_pct.mean() * 252) / (downside_std * np.sqrt(252)) if downside_std > 0 else 0

            best_day = daily_returns.max()
            best_day_date = daily_returns.idxmax()
        else:
            sharpe = 0
            sortino = 0
            best_day = 0
            best_day_date = None

        # Vectorized streak calculation
        trades_df['win'] = trades_df['monetary_pnl'] > 0
        trades_df['streak'] = (trades_df['win'] != trades_df['win'].shift()).cumsum()
        win_streaks = trades_df[trades_df['win']].groupby('streak').size()
        loss_streaks = trades_df[~trades_df['win']].groupby('streak').size()

        max_consecutive_wins = win_streaks.max() if len(win_streaks) > 0 else 0
        max_consecutive_losses = loss_streaks.max() if len(loss_streaks) > 0 else 0

        return {
            'total_trades': total_trades,
            'winning_trades': winning_trades,
            'losing_trades': losing_trades,
            'win_rate_%': round(win_rate, 2),
            'total_pnl_$': round(total_pnl, 2),
            'total_pips': round(total_pips, 1),
            'avg_win_$': round(avg_win, 2),
            'avg_loss_$': round(avg_loss, 2),
            'largest_win_$': round(largest_win, 2),
            'largest_loss_$': round(largest_loss, 2),
            'profit_factor': round(profit_factor, 2),
            'max_drawdown_%': round(max_drawdown_pct, 2),
            'sharpe_ratio': round(sharpe, 2),
            'sortino_ratio': round(sortino, 2),
            'total_return_%': round(total_return, 2),
            'final_balance_$': round(final_balance, 2),
            'best_day_$': round(best_day, 2) if best_day_date else 0,
            'best_day_date': str(best_day_date) if best_day_date else 'N/A',
            'max_consecutive_wins': int(max_consecutive_wins),
            'max_consecutive_losses': int(max_consecutive_losses),
            'avg_time_in_trade_hrs': round(trades_df['time_in_trade_hours'].mean(), 2)
        }


# ======================================================================
# 5. PARALLEL STRATEGY RUNNER
# ======================================================================
def run_single_strategy(args):
    """Worker function for parallel strategy execution"""
    df, strategy_name, strategy_func, sl_pips, tp_pips, pair_name, initial_balance, \
    leverage, risk_percent, spread_pips, slippage_pips, pip_value = args
    
    try:
        backtester = EnhancedBacktester(
            df,
            initial_balance=initial_balance,
            pip_value=pip_value,
            leverage=leverage,
            risk_percent=risk_percent,
            spread_pips=spread_pips,
            slippage_pips=slippage_pips
        )
        
        summary, metrics = backtester.run_backtest(
            strategy_func,
            sl_pips,
            tp_pips,
            pair_name,
            progress_callback=None  # No callback in parallel mode
        )
        
        return {
            'strategy_name': strategy_name,
            'summary': summary,
            'metrics': metrics,
            'trades_df': backtester.results,
            'success': True,
            'error': None
        }
    except Exception as e:
        logging.error(f"Error in strategy {strategy_name}: {e}")
        return {
            'strategy_name': strategy_name,
            'summary': f"Error: {str(e)}",
            'metrics': {},
            'trades_df': pd.DataFrame(),
            'success': False,
            'error': str(e)
        }


# ======================================================================
# 6. STRATEGY EDITOR WINDOW
# ======================================================================
class StrategyEditorWindow:
    """Popup window for editing strategy code"""
    
    def __init__(self, parent, strategy_name, strategy_func):
        self.window = ctk.CTkToplevel(parent)
        self.window.title(f"Edit Strategy: {strategy_name}")
        self.window.geometry("900x700")
        
        self.strategy_name = strategy_name
        self.strategy_func = strategy_func
        self.code_modified = False
        
        self.setup_ui()
        
    def setup_ui(self):
        """Create editor interface"""
        # Header
        header = ctk.CTkFrame(self.window, fg_color="#000000", height=60)
        header.pack(fill='x', padx=0, pady=0)
        
        ctk.CTkLabel(
            header,
            text=f"✏️  {self.strategy_name}",
            font=ctk.CTkFont(family="Helvetica", size=16, weight="bold"),
            text_color="#e6edf3"
        ).pack(side='left', padx=20, pady=15)
        
        # Code editor
        editor_frame = ctk.CTkFrame(self.window, fg_color="#0d1117")
        editor_frame.pack(fill='both', expand=True, padx=20, pady=(0, 10))
        
        # Get source code
        source = inspect.getsource(self.strategy_func)
        
        # Create text widget with dark theme
        self.text_editor = scrolledtext.ScrolledText(
            editor_frame,
            wrap=tk.WORD,
            font=("Courier New", 10),
            bg="#0d1117",
            fg="#e6edf3",
            insertbackground="#e6edf3",
            selectbackground="#388bfd",
            selectforeground="#ffffff",
            relief=tk.FLAT,
            borderwidth=0,
            padx=10,
            pady=10
        )
        self.text_editor.pack(fill='both', expand=True, padx=2, pady=2)
        self.text_editor.insert('1.0', source)
        
        # Buttons
        button_frame = ctk.CTkFrame(self.window, fg_color="transparent")
        button_frame.pack(fill='x', padx=20, pady=(0, 20))
        
        ctk.CTkButton(
            button_frame,
            text="💾 Save Changes",
            command=self.save_changes,
            fg_color="#238636",
            hover_color="#2ea043",
            height=38,
            font=ctk.CTkFont(family="Helvetica", size=12, weight="bold")
        ).pack(side='left', padx=(0, 10))
        
        ctk.CTkButton(
            button_frame,
            text="↺ Reset to Default",
            command=self.reset_code,
            fg_color="#21262d",
            hover_color="#30363d",
            height=38,
            font=ctk.CTkFont(family="Helvetica", size=12)
        ).pack(side='left', padx=(0, 10))
        
        ctk.CTkButton(
            button_frame,
            text="✕ Cancel",
            command=self.window.destroy,
            fg_color="#21262d",
            hover_color="#30363d",
            height=38,
            font=ctk.CTkFont(family="Helvetica", size=12)
        ).pack(side='right')
        
    def save_changes(self):
        """Save edited code"""
        try:
            new_code = self.text_editor.get('1.0', tk.END)
            
            # Validate syntax
            compile(new_code, '<string>', 'exec')
            
            # Execute code in TradingStrategies namespace
            exec_globals = {'np': np, 'pd': pd}
            exec(new_code, exec_globals)
            
            # Find the function
            func_name = self.strategy_func.__name__
            if func_name in exec_globals:
                # Update the function
                setattr(TradingStrategies, func_name, staticmethod(exec_globals[func_name]))
                self.code_modified = True
                
                messagebox.showinfo(
                    "Success",
                    f"Strategy '{self.strategy_name}' updated successfully!"
                )
                self.window.destroy()
            else:
                messagebox.showerror(
                    "Error",
                    f"Function '{func_name}' not found in code"
                )
                
        except SyntaxError as e:
            messagebox.showerror("Syntax Error", f"Invalid Python syntax:\n{str(e)}")
        except Exception as e:
            messagebox.showerror("Error", f"Failed to save strategy:\n{str(e)}")
            
    def reset_code(self):
        """Reset to original code"""
        if messagebox.askyesno("Reset Code", "Reset to default strategy code?"):
            source = inspect.getsource(self.strategy_func)
            self.text_editor.delete('1.0', tk.END)
            self.text_editor.insert('1.0', source)


# Continue in next file due to length...
# Part 2: UI and Report Generation
# Append this to algohaus_backtester_v7_enhanced.py or run as continuation

# (Continued from previous file...)

# Import the HTMLReportGenerator from your original file
# For now, I'll create a placeholder - you'll merge with your existing HTMLReportGenerator

class HTMLReportGenerator:
    """Simplified - merge with your existing full implementation"""
    
    @staticmethod
    def generate_report(metrics, trades_df, strategy_name, timeframe, pair, initial_balance,
                       leverage, sl_pips, tp_pips, risk_pct, start_date, end_date, df=None, output_dir=None):
        """Generate HTML report - use your existing implementation"""
        # This is a placeholder - use your full HTMLReportGenerator implementation
        timestamp = datetime.now().strftime("%Y-%m-%d_%H%M%S")
        filename = f"AlgoHaus_{pair.replace('/', '-')}_{strategy_name}_{timestamp}.html"
        
        if output_dir and os.path.exists(output_dir):
            report_path = os.path.join(output_dir, filename)
        else:
            report_path = os.path.join(tempfile.gettempdir(), filename)
            
        # Use your existing HTML generation here
        # For now, creating simple placeholder
        html = f"""
<!DOCTYPE html>
<html>
<head><title>AlgoHaus Report - {strategy_name}</title></head>
<body style="background: #0a0a0a; color: #fff; font-family: Arial; padding: 20px;">
<h1>AlgoHaus Backtest Report</h1>
<h2>{pair} - {strategy_name}</h2>
<p>Full report generation - integrate your HTMLReportGenerator here</p>
</body>
</html>
"""
        
        with open(report_path, 'w', encoding='utf-8') as f:
            f.write(html)
            
        return report_path


# ══════════════════════════════════════════════════════════════════════
# 7. ENHANCED BACKTESTER UI WITH MULTI-STRATEGY SUPPORT
# ══════════════════════════════════════════════════════════════════════
class BacktesterUI:
    def __init__(self, master):
        self.master = master
        master.title("⚡ AlgoHaus Backtester v7.0 - Multi-Strategy Edition")

        # Responsive window sizing
        screen_width = master.winfo_screenwidth()
        screen_height = master.winfo_screenheight()
        
        if screen_width >= 1920:
            width_ratio = 0.75
            height_ratio = 0.85
        elif screen_width >= 1366:
            width_ratio = 0.85
            height_ratio = 0.90
        else:
            width_ratio = 0.95
            height_ratio = 0.95
        
        window_width = int(screen_width * width_ratio)
        window_height = int(screen_height * height_ratio)
        
        window_width = max(window_width, 1200)
        window_height = max(window_height, 700)
        
        x = (screen_width - window_width) // 2
        y = (screen_height - window_height) // 2
        master.geometry(f"{window_width}x{window_height}+{x}+{y}")
        master.minsize(1000, 600)
        master.resizable(True, True)

        default_path = pathlib.Path(r"D:\compressedworld\AlgoHaus\OandaHistoricalData\1MinCharts")
        self.data_folder = default_path if default_path.exists() else pathlib.Path.cwd() / "data"
        self.df = None
        self.current_section = "config"

        # Strategy selection (CHANGED TO DICT)
        self.selected_strategies = {
            'vwap_crossover_strategy': tk.BooleanVar(master, value=True),
            'opening_range_strategy': tk.BooleanVar(master, value=False),
            'bollinger_band_reversion_strategy': tk.BooleanVar(master, value=False)
        }
        
        # Other settings
        self.selected_pair = tk.StringVar(master, value="EUR/USD")
        self.selected_timeframe = tk.StringVar(master, value="1hr")
        self.initial_balance = tk.DoubleVar(master, value=10000.0)
        self.leverage = tk.IntVar(master, value=50)
        self.sl_pips = tk.IntVar(master, value=30)
        self.tp_pips = tk.IntVar(master, value=60)
        self.risk_percent = tk.DoubleVar(master, value=1.0)
        self.spread_pips = tk.DoubleVar(master, value=1.5)
        self.slippage_pips = tk.DoubleVar(master, value=0.5)

        today = date.today()
        self.end_date_var = tk.StringVar(master, value=today.strftime("%Y-%m-%d"))
        self.start_date_var = tk.StringVar(master, value=(today - timedelta(days=365)).strftime("%Y-%m-%d"))

        self.status_text = tk.StringVar(master, value="Ready - Multi-Strategy Backtesting v7.0")
        
        # Multi-strategy results storage
        self.all_results = {}  # {strategy_name: {'metrics': {}, 'trades_df': df, 'summary': str}}
        self.summary_labels = {}

        self.setup_ui()
        self.refresh_available_pairs()
        self.master.after(500, self.update_pair_info)

        if self.data_folder.exists():
            self.update_status("Data folder ready • " + self.data_folder.name, "#238636")
        else:
            self.update_status("Data folder not found - please select", "#f85149")

    def setup_ui(self):
        """Setup UI with strategy checkboxes"""
        main_container = ctk.CTkFrame(self.master, corner_radius=0, fg_color="#000000")
        main_container.pack(fill='both', expand=True)

        # Sidebar
        self.sidebar_width = 240
        sidebar = ctk.CTkFrame(main_container, corner_radius=0, fg_color="#000000", width=self.sidebar_width)
        sidebar.pack(side='left', fill='y')
        sidebar.pack_propagate(False)

        # Logo
        logo_frame = ctk.CTkFrame(sidebar, corner_radius=0, fg_color="transparent")
        logo_frame.pack(fill='x', padx=16, pady=(20, 30))

        ctk.CTkLabel(
            logo_frame,
            text="⚡ algoHaus v7.0",
            font=ctk.CTkFont(family="Helvetica", size=20, weight="bold"),
            text_color="#6e7681",
            anchor="w"
        ).pack(anchor='w')

        ctk.CTkLabel(
            logo_frame,
            text="Multi-Strategy Backtester",
            font=ctk.CTkFont(family="Helvetica", size=10),
            text_color="#6e7681",
            anchor="w"
        ).pack(anchor='w', pady=(2, 0))

        # Navigation
        nav_frame = ctk.CTkFrame(sidebar, corner_radius=0, fg_color="transparent")
        nav_frame.pack(fill='x', padx=12, pady=(0, 20))

        self.nav_buttons = {}

        self.nav_buttons['config'] = self.create_nav_button(
            nav_frame, "⚙  Configuration", "config", selected=True
        )

        self.nav_buttons['strategy'] = self.create_nav_button(
            nav_frame, "📊 Strategies & Risk", "strategy"
        )

        self.nav_buttons['account'] = self.create_nav_button(
            nav_frame, "💰 Account", "account"
        )

        self.nav_buttons['results'] = self.create_nav_button(
            nav_frame, "📈 Results", "results"
        )

        # Run button
        self.run_btn = ctk.CTkButton(
            sidebar,
            text="▶  RUN BACKTESTS",
            font=ctk.CTkFont(family="Helvetica", size=13, weight="bold"),
            fg_color="#238636",
            hover_color="#2ea043",
            text_color="#ffffff",
            height=44,
            corner_radius=6,
            command=self.start_backtest_thread
        )
        self.run_btn.pack(fill='x', padx=12, pady=(15, 8))

        # Report button
        self.report_button = ctk.CTkButton(
            sidebar,
            text="📄 Generate Reports",
            font=ctk.CTkFont(family="Helvetica", size=12),
            fg_color="#21262d",
            hover_color="#388bfd",
            text_color="#ffffff",
            height=38,
            corner_radius=6,
            command=self.generate_all_reports,
            state="disabled"
        )
        self.report_button.pack(fill='x', padx=12, pady=(0, 8))

        # Utility buttons
        utility_container = ctk.CTkFrame(sidebar, fg_color="transparent")
        utility_container.pack(fill='x', padx=12, pady=(5, 15))

        ctk.CTkButton(
            utility_container,
            text="🗑️ Cache",
            font=ctk.CTkFont(family="Helvetica", size=10),
            fg_color="#21262d",
            hover_color="#30363d",
            text_color="#8b949e",
            height=32,
            corner_radius=6,
            command=self.clear_cache
        ).pack(side='left', expand=True, fill='x', padx=(0, 3))

        ctk.CTkButton(
            utility_container,
            text="📊 Export",
            font=ctk.CTkFont(family="Helvetica", size=10),
            fg_color="#21262d",
            hover_color="#30363d",
            text_color="#8b949e",
            height=32,
            corner_radius=6,
            command=self.export_all_to_csv
        ).pack(side='left', expand=True, fill='x', padx=(3, 0))

        # Content area
        content_area = ctk.CTkFrame(main_container, corner_radius=0, fg_color="#000000")
        content_area.pack(side='right', fill='both', expand=True)

        self.content_frame = ctk.CTkFrame(content_area, fg_color="transparent")
        self.content_frame.pack(fill='both', expand=True, padx=25, pady=20)

        self.sections = {}
        self.create_config_section()
        self.create_strategy_section()  # MODIFIED
        self.create_account_section()
        self.create_results_section()

        self.show_section('config')

        # Progress bar
        self.progress_container = ctk.CTkFrame(content_area, fg_color="#000000", corner_radius=8)

        self.progress_label = ctk.CTkLabel(
            self.progress_container,
            text="",
            font=ctk.CTkFont(family="Helvetica", size=10),
            text_color="#8b949e",
            anchor="w"
        )
        self.progress_label.pack(fill='x', padx=12, pady=(10, 5))

        self.progress_bar = ctk.CTkProgressBar(
            self.progress_container,
            mode="determinate",
            height=5,
            corner_radius=3,
            fg_color="#21262d",
            progress_color="#238636"
        )
        self.progress_bar.pack(fill='x', padx=12, pady=(0, 10))
        self.progress_bar.set(0)

        # Status bar
        status_bar = ctk.CTkFrame(self.master, corner_radius=0, height=32, fg_color="#000000")
        status_bar.pack(side='bottom', fill='x')

        self.status_label = ctk.CTkLabel(
            status_bar,
            textvariable=self.status_text,
            font=ctk.CTkFont(family="Helvetica", size=10),
            text_color="#8b949e",
            anchor="w"
        )
        self.status_label.pack(side='left', padx=25, pady=8)

    def create_nav_button(self, parent, text, section_id, selected=False):
        """Create navigation button"""
        btn = ctk.CTkButton(
            parent,
            text=text,
            font=ctk.CTkFont(family="Helvetica", size=12),
            fg_color="#21262d" if selected else "transparent",
            hover_color="#30363d" if selected else "#21262d",
            text_color="#e6edf3" if selected else "#8b949e",
            anchor="w",
            height=36,
            corner_radius=6,
            command=lambda: self.show_section(section_id)
        )
        btn.pack(fill='x', pady=2)
        return btn

    def update_nav_selection(self, selected_section):
        """Update navigation selection"""
        for section_id, btn in self.nav_buttons.items():
            if section_id == selected_section:
                btn.configure(fg_color="#21262d", text_color="#e6edf3")
            else:
                btn.configure(fg_color="transparent", text_color="#8b949e")

    def show_section(self, section_id):
        """Show specific section"""
        self.current_section = section_id
        self.update_nav_selection(section_id)

        for sec_id, sec_frame in self.sections.items():
            if sec_id == section_id:
                sec_frame.pack(fill='both', expand=True)
            else:
                sec_frame.pack_forget()

    def create_config_section(self):
        """Configuration section - same as original"""
        section = ctk.CTkFrame(self.content_frame, fg_color="transparent")
        self.sections['config'] = section

        ctk.CTkLabel(
            section,
            text="Configuration",
            font=ctk.CTkFont(family="Helvetica", size=16, weight="normal"),
            text_color="#e6edf3",
            anchor="w"
        ).pack(anchor='w', pady=(0, 20))

        content = ctk.CTkFrame(section, fg_color="#000000", corner_radius=10)
        content.pack(fill='both', expand=True)

        inner = ctk.CTkFrame(content, fg_color="transparent")
        inner.pack(fill='both', expand=True, padx=25, pady=25)

        # Data Folder
        self.create_section_header(inner, "Data Source")
        folder_frame = ctk.CTkFrame(inner, fg_color="#21262d", corner_radius=8)
        folder_frame.pack(fill='x', pady=(0, 20))

        folder_inner = ctk.CTkFrame(folder_frame, fg_color="transparent")
        folder_inner.pack(fill='x', padx=12, pady=10)

        ctk.CTkLabel(
            folder_inner,
            text="Data Folder",
            font=ctk.CTkFont(family="Helvetica", size=11),
            text_color="#8b949e",
            width=90,
            anchor="w"
        ).pack(side='left')

        folder_display = str(self.data_folder)
        if len(folder_display) > 45:
            folder_display = "..." + folder_display[-42:]

        self.folder_label = ctk.CTkLabel(
            folder_inner,
            text=folder_display,
            font=ctk.CTkFont(family="Helvetica", size=10),
            text_color="#e6edf3",
            anchor="w"
        )
        self.folder_label.pack(side='left', fill='x', expand=True, padx=12)

        ctk.CTkButton(
            folder_inner,
            text="Browse",
            command=self.select_data_folder,
            width=70,
            height=28,
            font=ctk.CTkFont(family="Helvetica", size=10),
            fg_color="#30363d",
            hover_color="#484f58",
            text_color="#e6edf3",
            corner_radius=6
        ).pack(side='right')

        # Trading Pair
        self.create_section_header(inner, "Trading Pair")
        self.pair_combo = self.create_sleek_input(inner, "Pair", self.selected_pair, is_combobox=True, values=["EUR/USD"])

        # Pair Info
        self.pair_info_frame = ctk.CTkFrame(inner, fg_color="#21262d", corner_radius=8)
        self.pair_info_frame.pack(fill='x', pady=(8, 20))

        self.pair_info_label = ctk.CTkLabel(
            self.pair_info_frame,
            text='Select a pair to view details...',
            font=ctk.CTkFont(family="Helvetica", size=10),
            text_color="#8b949e",
            anchor="nw",
            justify="left"
        )
        self.pair_info_label.pack(fill='both', padx=12, pady=12)

        self.selected_pair.trace('w', self.update_pair_info)

        # Timeframe & Dates
        self.create_section_header(inner, "Time Period")
        self.create_sleek_input(inner, "Timeframe", self.selected_timeframe, is_combobox=True,
                               values=["1min", "5min", "15min", "1hr", "1Day"])
        self.create_sleek_input(inner, "Start Date", self.start_date_var)
        self.create_sleek_input(inner, "End Date", self.end_date_var)

    def create_strategy_section(self):
        """MODIFIED: Strategy section with checkboxes and edit buttons"""
        section = ctk.CTkFrame(self.content_frame, fg_color="transparent")
        self.sections['strategy'] = section

        ctk.CTkLabel(
            section,
            text="Strategies & Risk",
            font=ctk.CTkFont(family="Helvetica", size=16, weight="normal"),
            text_color="#e6edf3",
            anchor="w"
        ).pack(anchor='w', pady=(0, 20))

        content = ctk.CTkFrame(section, fg_color="#000000", corner_radius=10)
        content.pack(fill='both', expand=True)

        inner = ctk.CTkFrame(content, fg_color="transparent")
        inner.pack(fill='both', expand=True, padx=25, pady=25)

        # Strategy Selection with Checkboxes
        self.create_section_header(inner, "Trading Strategies (Select Multiple)")
        
        strategy_container = ctk.CTkFrame(inner, fg_color="#21262d", corner_radius=8)
        strategy_container.pack(fill='x', pady=(0, 20))
        
        strategy_inner = ctk.CTkFrame(strategy_container, fg_color="transparent")
        strategy_inner.pack(fill='both', padx=12, pady=12)

        strategy_display_names = {
            'vwap_crossover_strategy': 'VWAP Crossover',
            'opening_range_strategy': 'Opening Range Breakout',
            'bollinger_band_reversion_strategy': 'Bollinger Band Mean Reversion'
        }

        for strategy_key, strategy_var in self.selected_strategies.items():
            # Create row for each strategy
            row = ctk.CTkFrame(strategy_inner, fg_color="transparent")
            row.pack(fill='x', pady=4)
            
            # Checkbox
            checkbox = ctk.CTkCheckBox(
                row,
                text=strategy_display_names[strategy_key],
                variable=strategy_var,
                font=ctk.CTkFont(family="Helvetica", size=11),
                text_color="#e6edf3",
                fg_color="#238636",
                hover_color="#2ea043",
                checkbox_width=20,
                checkbox_height=20
            )
            checkbox.pack(side='left', anchor='w')
            
            # Edit button
            edit_btn = ctk.CTkButton(
                row,
                text="✏️ Edit",
                width=70,
                height=28,
                font=ctk.CTkFont(family="Helvetica", size=10),
                fg_color="#30363d",
                hover_color="#484f58",
                text_color="#8b949e",
                corner_radius=6,
                command=lambda sk=strategy_key: self.edit_strategy(sk)
            )
            edit_btn.pack(side='right')

        # Select/Deselect All
        btn_row = ctk.CTkFrame(strategy_container, fg_color="transparent")
        btn_row.pack(fill='x', padx=12, pady=(0, 10))
        
        ctk.CTkButton(
            btn_row,
            text="✓ Select All",
            width=100,
            height=28,
            font=ctk.CTkFont(family="Helvetica", size=10),
            fg_color="#30363d",
            hover_color="#484f58",
            text_color="#8b949e",
            corner_radius=6,
            command=self.select_all_strategies
        ).pack(side='left', padx=(0, 5))
        
        ctk.CTkButton(
            btn_row,
            text="✗ Deselect All",
            width=100,
            height=28,
            font=ctk.CTkFont(family="Helvetica", size=10),
            fg_color="#30363d",
            hover_color="#484f58",
            text_color="#8b949e",
            corner_radius=6,
            command=self.deselect_all_strategies
        ).pack(side='left')

        # Risk Management
        self.create_section_header(inner, "Risk Management")
        self.create_sleek_input(inner, "Stop Loss (pips)", self.sl_pips)
        self.create_sleek_input(inner, "Take Profit (pips)", self.tp_pips)

        # Execution Costs
        self.create_section_header(inner, "Execution Costs")
        self.create_sleek_input(inner, "Spread (pips)", self.spread_pips)
        self.create_sleek_input(inner, "Slippage (pips)", self.slippage_pips)

    def edit_strategy(self, strategy_key):
        """Open strategy editor window"""
        strategy_func = getattr(TradingStrategies, strategy_key)
        display_name = {
            'vwap_crossover_strategy': 'VWAP Crossover',
            'opening_range_strategy': 'Opening Range Breakout',
            'bollinger_band_reversion_strategy': 'Bollinger Band Mean Reversion'
        }[strategy_key]
        
        StrategyEditorWindow(self.master, display_name, strategy_func)

    def select_all_strategies(self):
        """Select all strategies"""
        for var in self.selected_strategies.values():
            var.set(True)

    def deselect_all_strategies(self):
        """Deselect all strategies"""
        for var in self.selected_strategies.values():
            var.set(False)

    def create_account_section(self):
        """Account section - same as original"""
        section = ctk.CTkFrame(self.content_frame, fg_color="transparent")
        self.sections['account'] = section

        ctk.CTkLabel(
            section,
            text="Account Settings",
            font=ctk.CTkFont(family="Helvetica", size=16, weight="normal"),
            text_color="#e6edf3",
            anchor="w"
        ).pack(anchor='w', pady=(0, 20))

        content = ctk.CTkFrame(section, fg_color="#000000", corner_radius=10)
        content.pack(fill='both', expand=True)

        inner = ctk.CTkFrame(content, fg_color="transparent")
        inner.pack(fill='both', expand=True, padx=25, pady=25)

        self.create_section_header(inner, "Capital & Leverage")
        self.create_sleek_input(inner, "Initial Balance ($)", self.initial_balance)
        self.create_sleek_input(inner, "Leverage", self.leverage, is_combobox=True,
                               values=[str(x) for x in ForexCalculator.LEVERAGE_OPTIONS])

        self.create_section_header(inner, "Position Sizing")
        self.create_sleek_input(inner, "Risk % per Trade", self.risk_percent)

    def create_results_section(self):
        """Results section with multi-strategy support"""
        section = ctk.CTkFrame(self.content_frame, fg_color="transparent")
        self.sections['results'] = section

        # Main scrollable container
        self.results_scroll = ctk.CTkScrollableFrame(
            section,
            fg_color="transparent",
            scrollbar_button_color="#30363d",
            scrollbar_button_hover_color="#484f58"
        )
        self.results_scroll.pack(fill='both', expand=True)

        # Header
        ctk.CTkLabel(
            self.results_scroll,
            text="Multi-Strategy Results",
            font=ctk.CTkFont(family="Helvetica", size=16, weight="normal"),
            text_color="#e6edf3",
            anchor="w"
        ).pack(anchor='w', pady=(0, 15))

        # Results will be populated dynamically
        self.results_container = ctk.CTkFrame(self.results_scroll, fg_color="transparent")
        self.results_container.pack(fill='both', expand=True)

    # Helper methods
    def create_section_header(self, parent, text):
        """Create section header"""
        ctk.CTkLabel(
            parent,
            text=text,
            font=ctk.CTkFont(family="Helvetica", size=12, weight="bold"),
            text_color="#8b949e",
            anchor="w"
        ).pack(anchor='w', pady=(12, 6))

    def create_sleek_input(self, parent, label_text, variable, is_combobox=False, values=None):
        """Create input field"""
        frame = ctk.CTkFrame(parent, fg_color="transparent")
        frame.pack(fill='x', pady=5)

        ctk.CTkLabel(
            frame,
            text=label_text,
            font=ctk.CTkFont(family="Helvetica", size=11),
            text_color="#8b949e",
            width=120,
            anchor="w"
        ).pack(side='left')

        if is_combobox:
            widget = ctk.CTkComboBox(
                frame,
                variable=variable,
                values=values or [],
                font=ctk.CTkFont(family="Helvetica", size=11),
                fg_color="#21262d",
                border_color="#30363d",
                button_color="#30363d",
                button_hover_color="#484f58",
                text_color="#e6edf3",
                dropdown_fg_color="#000000",
                dropdown_hover_color="#21262d",
                dropdown_text_color="#e6edf3",
                corner_radius=6,
                height=32,
                width=240
            )
        else:
            widget = ctk.CTkEntry(
                frame,
                textvariable=variable,
                font=ctk.CTkFont(family="Helvetica", size=11),
                fg_color="#21262d",
                border_color="#30363d",
                text_color="#e6edf3",
                corner_radius=6,
                height=32,
                width=240
            )

        widget.pack(side='left', padx=(12, 0))
        return widget

    def update_status(self, text, color="#8b949e"):
        """Update status bar"""
        self.status_text.set(text)
        self.status_label.configure(text_color=color)

    def get_desktop_path(self):
        """Get desktop path"""
        desktop_path = os.path.join(os.path.expanduser("~"), "Desktop")
        if os.path.exists(desktop_path):
            return desktop_path

        onedrive_desktop = os.path.join(os.path.expanduser("~"), "OneDrive", "Desktop")
        if os.path.exists(onedrive_desktop):
            return onedrive_desktop

        if os.name == 'nt':
            try:
                import winreg
                key = winreg.OpenKey(winreg.HKEY_CURRENT_USER,
                                    r'Software\Microsoft\Windows\CurrentVersion\Explorer\Shell Folders')
                desktop = winreg.QueryValueEx(key, 'Desktop')[0]
                winreg.CloseKey(key)
                if desktop and os.path.exists(desktop):
                    return desktop
            except:
                pass

        return os.path.expanduser("~")

    def select_data_folder(self):
        """Select data folder"""
        new_folder = filedialog.askdirectory(
            title="Select Main Data Folder",
            initialdir=str(self.data_folder)
        )
        if new_folder:
            self.data_folder = pathlib.Path(new_folder)
            folder_text = str(self.data_folder)
            if len(folder_text) > 45:
                folder_text = "..." + folder_text[-42:]
            self.folder_label.configure(text=folder_text)
            self.refresh_available_pairs()
            self.update_status(f"Data folder updated", "#238636")

    def refresh_available_pairs(self):
        """Refresh available pairs"""
        try:
            self.update_status("Scanning for pairs...", "#8b949e")
            pairs = detect_available_pairs(self.data_folder)

            if pairs:
                if hasattr(self, 'pair_combo') and self.pair_combo:
                    self.pair_combo.configure(values=pairs)
                    if self.selected_pair.get() not in pairs:
                        self.selected_pair.set(pairs[0])
                self.update_status(f"Found {len(pairs)} pairs", "#3fb950")
            else:
                self.update_status("No pairs found", "#f85149")
        except Exception as e:
            logging.error(f"Error refreshing pairs: {e}")
            self.update_status(f"Error: {str(e)}", "#f85149")

    def update_pair_info(self, *args):
        """Update pair info display"""
        pair = self.selected_pair.get()
        if not pair:
            return

        start, end = get_data_date_range(pair, self.data_folder)
        pip_value = ForexCalculator.PIP_VALUES.get(pair, 0.0001)

        if start and end:
            total_days = (end - start).days
            info_text = f"""PAIR: {pair}

Data Available: {start} to {end}
Total Days: {total_days:,}

Pip Value: {pip_value}"""

            self.start_date_var.set(str(start))
            self.end_date_var.set(str(end))
            self.update_status(f"{pair}: {total_days:,} days of data", "#238636")
        else:
            info_text = f"PAIR: {pair}\n\nNo data found"
            self.update_status(f"No data found for {pair}", "#f85149")

        self.pair_info_label.configure(text=info_text)

    # PARALLEL BACKTEST EXECUTION
    def start_backtest_thread(self):
        """Start multi-strategy backtest"""
        # Check if any strategy is selected
        selected = [name for name, var in self.selected_strategies.items() if var.get()]
        
        if not selected:
            messagebox.showwarning("No Strategy", "Please select at least one strategy")
            return
        
        self.update_status(f"Running {len(selected)} strategies in parallel...", "#238636")
        
        # Clear previous results
        self.all_results = {}
        for widget in self.results_container.winfo_children():
            widget.destroy()
        
        self.report_button.configure(state="disabled")
        
        self.progress_container.pack(fill='x', padx=25, pady=(0, 15), before=self.content_frame)
        self.progress_bar.set(0)
        self.progress_label.configure(text="Initializing backtests...")

        try:
            start_date = datetime.strptime(self.start_date_var.get(), "%Y-%m-%d")
            end_date = datetime.strptime(self.end_date_var.get(), "%Y-%m-%d")
            if start_date >= end_date:
                raise ValueError("Start date must be before End date.")
        except ValueError as e:
            messagebox.showerror("Input Error", f"Invalid date input: {e}")
            self.update_status("Invalid date input", "#f85149")
            self.progress_container.pack_forget()
            return

        self.q = queue.Queue()
        threading.Thread(target=self.run_parallel_backtests,
                         args=(start_date, end_date, selected),
                         daemon=True).start()
        self.master.after(100, self.check_queue)

    def run_parallel_backtests(self, start_date, end_date, selected_strategies):
        """Run multiple strategies in parallel"""
        try:
            pair = self.selected_pair.get()
            timeframe = self.selected_timeframe.get()
            
            # Load data once
            self.q.put(('progress', 0, f"Loading data for {pair}..."))
            df, actual_start, actual_end = load_pair_data(pair, self.data_folder, start_date, end_date, timeframe)
            self.df = df
            
            # Prepare strategy arguments
            strategy_args = []
            for strategy_name in selected_strategies:
                strategy_func = getattr(TradingStrategies, strategy_name)
                
                args = (
                    df.copy(),  # Each strategy gets its own copy
                    strategy_name,
                    strategy_func,
                    self.sl_pips.get(),
                    self.tp_pips.get(),
                    pair,
                    self.initial_balance.get(),
                    self.leverage.get(),
                    self.risk_percent.get(),
                    self.spread_pips.get(),
                    self.slippage_pips.get(),
                    ForexCalculator.PIP_VALUES.get(pair, 0.0001)
                )
                strategy_args.append(args)
            
            # Run strategies in parallel using ThreadPoolExecutor (faster for I/O)
            results = []
            with ThreadPoolExecutor(max_workers=min(len(selected_strategies), 4)) as executor:
                futures = {executor.submit(run_single_strategy, args): args[1] for args in strategy_args}
                
                completed = 0
                for future in as_completed(futures):
                    result = future.result()
                    results.append(result)
                    completed += 1
                    progress = int((completed / len(selected_strategies)) * 100)
                    self.q.put(('progress', progress, f"Completed {completed}/{len(selected_strategies)} strategies"))
            
            self.q.put(('success', results))
            
        except Exception as e:
            logging.error(f"Parallel backtest error: {e}", exc_info=True)
            self.q.put(('error', str(e)))

    def check_queue(self):
        """Check for backtest results"""
        try:
            result_type, *data = self.q.get_nowait()

            if result_type == 'progress':
                percent, message = data
                self.progress_bar.set(percent / 100)
                self.progress_label.configure(text=message)
                self.master.after(100, self.check_queue)
            elif result_type == 'success':
                results = data[0]
                self.display_multi_strategy_results(results)
                self.update_status(f"Completed {len(results)} strategies", "#3fb950")
                self.progress_bar.set(1.0)
                self.progress_label.configure(text="All backtests complete!")
                self.master.after(2000, self.progress_container.pack_forget)
            elif result_type == 'error':
                error_msg = data[0]
                messagebox.showerror("Backtest Error", error_msg)
                self.update_status("Backtest failed", "#f85149")
                self.progress_container.pack_forget()

        except queue.Empty:
            self.master.after(100, self.check_queue)

    def display_multi_strategy_results(self, results):
        """Display results from multiple strategies"""
        # Clear container
        for widget in self.results_container.winfo_children():
            widget.destroy()
        
        # Store results
        for result in results:
            if result['success']:
                self.all_results[result['strategy_name']] = result
        
        if not self.all_results:
            ctk.CTkLabel(
                self.results_container,
                text="No successful strategy results",
                font=ctk.CTkFont(family="Helvetica", size=14),
                text_color="#f85149"
            ).pack(pady=50)
            return
        
        # Display each strategy's results
        for idx, (strategy_name, result) in enumerate(self.all_results.items()):
            self.create_strategy_result_card(strategy_name, result, idx)
        
        self.report_button.configure(state="normal")
        self.show_section('results')

    def create_strategy_result_card(self, strategy_name, result, index):
        """Create result card for a single strategy"""
        display_names = {
            'vwap_crossover_strategy': 'VWAP Crossover',
            'opening_range_strategy': 'Opening Range Breakout',
            'bollinger_band_reversion_strategy': 'Bollinger Band Mean Reversion'
        }
        
        display_name = display_names.get(strategy_name, strategy_name)
        
        # Card container
        card = ctk.CTkFrame(self.results_container, fg_color="#0d1117", corner_radius=10)
        card.pack(fill='x', pady=(0 if index == 0 else 10, 0))
        
        # Header
        header = ctk.CTkFrame(card, fg_color="#161b22", corner_radius=8)
        header.pack(fill='x', padx=15, pady=15)
        
        ctk.CTkLabel(
            header,
            text=f"📊 {display_name}",
            font=ctk.CTkFont(family="Helvetica", size=14, weight="bold"),
            text_color="#e6edf3",
            anchor="w"
        ).pack(side='left', padx=12, pady=10)
        
        # Summary metrics
        metrics = result['metrics']
        trades_df = result['trades_df']
        
        if not trades_df.empty:
            summary_frame = ctk.CTkFrame(card, fg_color="transparent")
            summary_frame.pack(fill='x', padx=15, pady=(0, 15))
            
            # Configure grid
            for i in range(6):
                summary_frame.columnconfigure(i, weight=1)
            
            # Create metric cards
            total_trades = len(trades_df)
            win_rate = metrics.get('win_rate_%', 0)
            total_pnl = metrics.get('total_pnl_$', 0)
            total_return = metrics.get('total_return_%', 0)
            sharpe = metrics.get('sharpe_ratio', 0)
            max_dd = metrics.get('max_drawdown_%', 0)
            
            metric_data = [
                ('Trades', str(total_trades), '#e6edf3'),
                ('Win Rate', f"{win_rate:.1f}%", '#3fb950' if win_rate >= 50 else '#f85149'),
                ('Total P&L', f"${total_pnl:,.0f}", '#3fb950' if total_pnl >= 0 else '#f85149'),
                ('Returns', f"{total_return:+.2f}%", '#3fb950' if total_return >= 0 else '#f85149'),
                ('Sharpe', f"{sharpe:.2f}", '#3fb950' if sharpe > 1 else '#f85149'),
                ('Max DD', f"{max_dd:.1f}%", '#f85149')
            ]
            
            for col, (label, value, color) in enumerate(metric_data):
                metric_card = ctk.CTkFrame(summary_frame, fg_color="#161b22", corner_radius=6)
                metric_card.grid(row=0, column=col, sticky='nsew', padx=4, pady=4)
                
                inner = ctk.CTkFrame(metric_card, fg_color="transparent")
                inner.pack(fill='both', expand=True, padx=8, pady=6)
                
                ctk.CTkLabel(
                    inner,
                    text=label,
                    font=ctk.CTkFont(family="Helvetica", size=9),
                    text_color="#6e7681",
                    anchor="w"
                ).pack(anchor='w')
                
                ctk.CTkLabel(
                    inner,
                    text=value,
                    font=ctk.CTkFont(family="Helvetica", size=13, weight="bold"),
                    text_color=color,
                    anchor="w"
                ).pack(anchor='w', pady=(2, 0))

    def generate_all_reports(self):
        """Generate reports for all strategies"""
        if not self.all_results:
            messagebox.showwarning("No Data", "No results available")
            return
        
        try:
            desktop = self.get_desktop_path()
            
            for strategy_name, result in self.all_results.items():
                if not result['success'] or result['trades_df'].empty:
                    continue
                
                report_path = HTMLReportGenerator.generate_report(
                    result['metrics'],
                    result['trades_df'],
                    strategy_name,
                    self.selected_timeframe.get(),
                    self.selected_pair.get(),
                    self.initial_balance.get(),
                    self.leverage.get(),
                    self.sl_pips.get(),
                    self.tp_pips.get(),
                    self.risk_percent.get(),
                    self.start_date_var.get(),
                    self.end_date_var.get(),
                    df=self.df,
                    output_dir=desktop
                )
                
                logging.info(f"Generated report: {report_path}")
            
            messagebox.showinfo("Success", f"Generated {len(self.all_results)} reports!\nSaved to Desktop")
            self.update_status("Reports generated", "#3fb950")
            
        except Exception as e:
            messagebox.showerror("Report Error", f"Failed:\n{str(e)}")
            self.update_status("Report error", "#f85149")

    def export_all_to_csv(self):
        """Export all strategy results to CSV"""
        if not self.all_results:
            messagebox.showwarning("No Data", "No results available")
            return
        
        try:
            desktop = self.get_desktop_path()
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            
            for strategy_name, result in self.all_results.items():
                if not result['success'] or result['trades_df'].empty:
                    continue
                
                filename = f"AlgoHaus_{strategy_name}_{timestamp}.csv"
                filepath = os.path.join(desktop, filename)
                
                result['trades_df'].to_csv(filepath, index=False)
                logging.info(f"Exported: {filepath}")
            
            messagebox.showinfo("Success", f"Exported {len(self.all_results)} CSV files to Desktop")
            self.update_status("Data exported", "#3fb950")
            
        except Exception as e:
            messagebox.showerror("Export Error", f"Failed:\n{str(e)}")
            self.update_status("Export failed", "#f85149")

    def clear_cache(self):
        """Clear data cache"""
        global _DATA_CACHE
        with _CACHE_LOCK:
            cache_size = len(_DATA_CACHE)
            _DATA_CACHE.clear()
        
        messagebox.showinfo("Cache Cleared", f"Cleared {cache_size} cached datasets")
        self.update_status("Cache cleared", "#3fb950")


# ══════════════════════════════════════════════════════════════════════
# 8. MAIN
# ══════════════════════════════════════════════════════════════════════
if __name__ == '__main__':
    # Enable multiprocessing support on Windows
    if os.name == 'nt':
        mp.freeze_support()
    
    app = ctk.CTk()
    backtester = BacktesterUI(app)
    app.mainloop()

In [2]:
# AlgoHaus Backtester v7.0 - COMPLETE UI with Charts and Metrics
# This replaces algohaus_backtester_v7_ui.py with FULL results display

# ══════════════════════════════════════════════════════════════════════
# IMPORTS - Add to the existing imports from part 1
# ══════════════════════════════════════════════════════════════════════

import customtkinter as ctk
import tkinter as tk
from tkinter import messagebox, filedialog, scrolledtext
import pandas as pd
import numpy as np
from datetime import datetime, timedelta, date
import webbrowser
import os
import sys
import tempfile
import inspect
import pathlib
import threading
import queue
import logging
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from matplotlib.figure import Figure
import seaborn as sns
import json
import base64
from io import BytesIO
from concurrent.futures import ThreadPoolExecutor, as_completed
import multiprocessing as mp

# Import from part 1 file
# from algohaus_backtester_v7_enhanced import (
#     ForexCalculator, TradingStrategies, EnhancedBacktester,
#     detect_available_pairs, get_data_date_range, load_pair_data,
#     run_single_strategy, StrategyEditorWindow
# )

ctk.set_appearance_mode("dark")
ctk.set_default_color_theme("green")


# ══════════════════════════════════════════════════════════════════════
# PLACEHOLDER - Use your existing HTMLReportGenerator
# ══════════════════════════════════════════════════════════════════════
class HTMLReportGenerator:
    """Placeholder - merge with your existing implementation"""
    
    @staticmethod
    def generate_report(metrics, trades_df, strategy_name, timeframe, pair, initial_balance,
                       leverage, sl_pips, tp_pips, risk_pct, start_date, end_date, df=None, output_dir=None):
        timestamp = datetime.now().strftime("%Y-%m-%d_%H%M%S")
        filename = f"AlgoHaus_{pair.replace('/', '-')}_{strategy_name}_{timestamp}.html"
        
        if output_dir and os.path.exists(output_dir):
            report_path = os.path.join(output_dir, filename)
        else:
            report_path = os.path.join(tempfile.gettempdir(), filename)
        
        # Use your full HTML generation here
        html = f"""<!DOCTYPE html>
<html><head><title>AlgoHaus Report - {strategy_name}</title></head>
<body style="background: #0a0a0a; color: #fff; font-family: Arial; padding: 20px;">
<h1>AlgoHaus Backtest Report</h1><h2>{pair} - {strategy_name}</h2>
<p>Integrate your full HTMLReportGenerator here</p></body></html>"""
        
        with open(report_path, 'w', encoding='utf-8') as f:
            f.write(html)
        return report_path


# ══════════════════════════════════════════════════════════════════════
# ENHANCED BACKTESTER UI WITH COMPLETE RESULTS VISUALIZATION
# ══════════════════════════════════════════════════════════════════════
class BacktesterUI:
    def __init__(self, master):
        self.master = master
        master.title("⚡ AlgoHaus Backtester v7.0 - Multi-Strategy Edition")

        # Responsive window sizing
        screen_width = master.winfo_screenwidth()
        screen_height = master.winfo_screenheight()
        
        if screen_width >= 1920:
            width_ratio = 0.75
            height_ratio = 0.85
        elif screen_width >= 1366:
            width_ratio = 0.85
            height_ratio = 0.90
        else:
            width_ratio = 0.95
            height_ratio = 0.95
        
        window_width = int(screen_width * width_ratio)
        window_height = int(screen_height * height_ratio)
        window_width = max(window_width, 1200)
        window_height = max(window_height, 700)
        
        x = (screen_width - window_width) // 2
        y = (screen_height - window_height) // 2
        master.geometry(f"{window_width}x{window_height}+{x}+{y}")
        master.minsize(1000, 600)
        master.resizable(True, True)

        # Assume you have a default data path or will browse
        default_path = pathlib.Path(r"D:\compressedworld\AlgoHaus\OandaHistoricalData\1MinCharts")
        self.data_folder = default_path if default_path.exists() else pathlib.Path.cwd() / "data"
        self.df = None
        self.current_section = "config"

        # Strategy selection (CHANGED TO DICT WITH CHECKBOXES)
        self.selected_strategies = {
            'vwap_crossover_strategy': tk.BooleanVar(master, value=True),
            'opening_range_strategy': tk.BooleanVar(master, value=False),
            'bollinger_band_reversion_strategy': tk.BooleanVar(master, value=False)
        }
        
        # Other settings
        self.selected_pair = tk.StringVar(master, value="EUR/USD")
        self.selected_timeframe = tk.StringVar(master, value="1hr")
        self.initial_balance = tk.DoubleVar(master, value=10000.0)
        self.leverage = tk.IntVar(master, value=50)
        self.sl_pips = tk.IntVar(master, value=30)
        self.tp_pips = tk.IntVar(master, value=60)
        self.risk_percent = tk.DoubleVar(master, value=1.0)
        self.spread_pips = tk.DoubleVar(master, value=1.5)
        self.slippage_pips = tk.DoubleVar(master, value=0.5)

        today = date.today()
        self.end_date_var = tk.StringVar(master, value=today.strftime("%Y-%m-%d"))
        self.start_date_var = tk.StringVar(master, value=(today - timedelta(days=365)).strftime("%Y-%m-%d"))

        self.status_text = tk.StringVar(master, value="Ready - Multi-Strategy Backtesting v7.0")
        
        # Multi-strategy results storage
        self.all_results = {}  # {strategy_name: {'metrics': {}, 'trades_df': df, 'summary': str}}

        self.setup_ui()
        
        # Only refresh pairs if using detect_available_pairs from part 1
        # self.refresh_available_pairs()
        # self.master.after(500, self.update_pair_info)

        if self.data_folder.exists():
            self.update_status("Data folder ready • " + self.data_folder.name, "#238636")
        else:
            self.update_status("Data folder not found - please select", "#f85149")

    def setup_ui(self):
        """Setup UI with strategy checkboxes"""
        main_container = ctk.CTkFrame(self.master, corner_radius=0, fg_color="#000000")
        main_container.pack(fill='both', expand=True)

        # Sidebar
        self.sidebar_width = 240
        sidebar = ctk.CTkFrame(main_container, corner_radius=0, fg_color="#000000", width=self.sidebar_width)
        sidebar.pack(side='left', fill='y')
        sidebar.pack_propagate(False)

        # Logo
        logo_frame = ctk.CTkFrame(sidebar, corner_radius=0, fg_color="transparent")
        logo_frame.pack(fill='x', padx=16, pady=(20, 30))

        ctk.CTkLabel(
            logo_frame,
            text="⚡ algoHaus v7.0",
            font=ctk.CTkFont(family="Helvetica", size=20, weight="bold"),
            text_color="#6e7681",
            anchor="w"
        ).pack(anchor='w')

        ctk.CTkLabel(
            logo_frame,
            text="Multi-Strategy Backtester",
            font=ctk.CTkFont(family="Helvetica", size=10),
            text_color="#6e7681",
            anchor="w"
        ).pack(anchor='w', pady=(2, 0))

        # Navigation
        nav_frame = ctk.CTkFrame(sidebar, corner_radius=0, fg_color="transparent")
        nav_frame.pack(fill='x', padx=12, pady=(0, 20))

        self.nav_buttons = {}
        self.nav_buttons['config'] = self.create_nav_button(nav_frame, "⚙  Configuration", "config", selected=True)
        self.nav_buttons['strategy'] = self.create_nav_button(nav_frame, "📊 Strategies & Risk", "strategy")
        self.nav_buttons['account'] = self.create_nav_button(nav_frame, "💰 Account", "account")
        self.nav_buttons['results'] = self.create_nav_button(nav_frame, "📈 Results", "results")

        # Run button
        self.run_btn = ctk.CTkButton(
            sidebar,
            text="▶  RUN BACKTESTS",
            font=ctk.CTkFont(family="Helvetica", size=13, weight="bold"),
            fg_color="#238636",
            hover_color="#2ea043",
            text_color="#ffffff",
            height=44,
            corner_radius=6,
            command=self.start_backtest_thread
        )
        self.run_btn.pack(fill='x', padx=12, pady=(15, 8))

        # Report button
        self.report_button = ctk.CTkButton(
            sidebar,
            text="📄 Generate Reports",
            font=ctk.CTkFont(family="Helvetica", size=12),
            fg_color="#21262d",
            hover_color="#388bfd",
            text_color="#ffffff",
            height=38,
            corner_radius=6,
            command=self.generate_all_reports,
            state="disabled"
        )
        self.report_button.pack(fill='x', padx=12, pady=(0, 8))

        # Utility buttons
        utility_container = ctk.CTkFrame(sidebar, fg_color="transparent")
        utility_container.pack(fill='x', padx=12, pady=(5, 15))

        ctk.CTkButton(
            utility_container,
            text="🗑️ Cache",
            font=ctk.CTkFont(family="Helvetica", size=10),
            fg_color="#21262d",
            hover_color="#30363d",
            text_color="#8b949e",
            height=32,
            corner_radius=6,
            command=self.clear_cache
        ).pack(side='left', expand=True, fill='x', padx=(0, 3))

        ctk.CTkButton(
            utility_container,
            text="📊 Export",
            font=ctk.CTkFont(family="Helvetica", size=10),
            fg_color="#21262d",
            hover_color="#30363d",
            text_color="#8b949e",
            height=32,
            corner_radius=6,
            command=self.export_all_to_csv
        ).pack(side='left', expand=True, fill='x', padx=(3, 0))

        # Content area
        content_area = ctk.CTkFrame(main_container, corner_radius=0, fg_color="#000000")
        content_area.pack(side='right', fill='both', expand=True)

        self.content_frame = ctk.CTkFrame(content_area, fg_color="transparent")
        self.content_frame.pack(fill='both', expand=True, padx=25, pady=20)

        self.sections = {}
        self.create_config_section()
        self.create_strategy_section()
        self.create_account_section()
        self.create_results_section()  # COMPLETE VERSION

        self.show_section('config')

        # Progress bar
        self.progress_container = ctk.CTkFrame(content_area, fg_color="#000000", corner_radius=8)

        self.progress_label = ctk.CTkLabel(
            self.progress_container,
            text="",
            font=ctk.CTkFont(family="Helvetica", size=10),
            text_color="#8b949e",
            anchor="w"
        )
        self.progress_label.pack(fill='x', padx=12, pady=(10, 5))

        self.progress_bar = ctk.CTkProgressBar(
            self.progress_container,
            mode="determinate",
            height=5,
            corner_radius=3,
            fg_color="#21262d",
            progress_color="#238636"
        )
        self.progress_bar.pack(fill='x', padx=12, pady=(0, 10))
        self.progress_bar.set(0)

        # Status bar
        status_bar = ctk.CTkFrame(self.master, corner_radius=0, height=32, fg_color="#000000")
        status_bar.pack(side='bottom', fill='x')

        self.status_label = ctk.CTkLabel(
            status_bar,
            textvariable=self.status_text,
            font=ctk.CTkFont(family="Helvetica", size=10),
            text_color="#8b949e",
            anchor="w"
        )
        self.status_label.pack(side='left', padx=25, pady=8)

    def create_nav_button(self, parent, text, section_id, selected=False):
        btn = ctk.CTkButton(
            parent,
            text=text,
            font=ctk.CTkFont(family="Helvetica", size=12),
            fg_color="#21262d" if selected else "transparent",
            hover_color="#30363d" if selected else "#21262d",
            text_color="#e6edf3" if selected else "#8b949e",
            anchor="w",
            height=36,
            corner_radius=6,
            command=lambda: self.show_section(section_id)
        )
        btn.pack(fill='x', pady=2)
        return btn

    def update_nav_selection(self, selected_section):
        for section_id, btn in self.nav_buttons.items():
            if section_id == selected_section:
                btn.configure(fg_color="#21262d", text_color="#e6edf3")
            else:
                btn.configure(fg_color="transparent", text_color="#8b949e")

    def show_section(self, section_id):
        self.current_section = section_id
        self.update_nav_selection(section_id)

        for sec_id, sec_frame in self.sections.items():
            if sec_id == section_id:
                sec_frame.pack(fill='both', expand=True)
            else:
                sec_frame.pack_forget()

    # ══════════════════════════════════════════════════════════════════
    # CONFIG SECTION
    # ══════════════════════════════════════════════════════════════════
    def create_config_section(self):
        section = ctk.CTkFrame(self.content_frame, fg_color="transparent")
        self.sections['config'] = section

        ctk.CTkLabel(
            section,
            text="Configuration",
            font=ctk.CTkFont(family="Helvetica", size=16, weight="normal"),
            text_color="#e6edf3",
            anchor="w"
        ).pack(anchor='w', pady=(0, 20))

        content = ctk.CTkFrame(section, fg_color="#000000", corner_radius=10)
        content.pack(fill='both', expand=True)

        inner = ctk.CTkFrame(content, fg_color="transparent")
        inner.pack(fill='both', expand=True, padx=25, pady=25)

        # Data Folder
        self.create_section_header(inner, "Data Source")
        folder_frame = ctk.CTkFrame(inner, fg_color="#21262d", corner_radius=8)
        folder_frame.pack(fill='x', pady=(0, 20))

        folder_inner = ctk.CTkFrame(folder_frame, fg_color="transparent")
        folder_inner.pack(fill='x', padx=12, pady=10)

        ctk.CTkLabel(
            folder_inner,
            text="Data Folder",
            font=ctk.CTkFont(family="Helvetica", size=11),
            text_color="#8b949e",
            width=90,
            anchor="w"
        ).pack(side='left')

        folder_display = str(self.data_folder)
        if len(folder_display) > 45:
            folder_display = "..." + folder_display[-42:]

        self.folder_label = ctk.CTkLabel(
            folder_inner,
            text=folder_display,
            font=ctk.CTkFont(family="Helvetica", size=10),
            text_color="#e6edf3",
            anchor="w"
        )
        self.folder_label.pack(side='left', fill='x', expand=True, padx=12)

        ctk.CTkButton(
            folder_inner,
            text="Browse",
            command=self.select_data_folder,
            width=70,
            height=28,
            font=ctk.CTkFont(family="Helvetica", size=10),
            fg_color="#30363d",
            hover_color="#484f58",
            text_color="#e6edf3",
            corner_radius=6
        ).pack(side='right')

        # Trading Pair
        self.create_section_header(inner, "Trading Pair")
        self.pair_combo = self.create_sleek_input(inner, "Pair", self.selected_pair, is_combobox=True, values=["EUR/USD"])

        # Timeframe & Dates
        self.create_section_header(inner, "Time Period")
        self.create_sleek_input(inner, "Timeframe", self.selected_timeframe, is_combobox=True,
                               values=["1min", "5min", "15min", "1hr", "1Day"])
        self.create_sleek_input(inner, "Start Date", self.start_date_var)
        self.create_sleek_input(inner, "End Date", self.end_date_var)

    # ══════════════════════════════════════════════════════════════════
    # STRATEGY SECTION WITH CHECKBOXES AND EDIT BUTTONS
    # ══════════════════════════════════════════════════════════════════
    def create_strategy_section(self):
        section = ctk.CTkFrame(self.content_frame, fg_color="transparent")
        self.sections['strategy'] = section

        ctk.CTkLabel(
            section,
            text="Strategies & Risk",
            font=ctk.CTkFont(family="Helvetica", size=16, weight="normal"),
            text_color="#e6edf3",
            anchor="w"
        ).pack(anchor='w', pady=(0, 20))

        content = ctk.CTkFrame(section, fg_color="#000000", corner_radius=10)
        content.pack(fill='both', expand=True)

        inner = ctk.CTkFrame(content, fg_color="transparent")
        inner.pack(fill='both', expand=True, padx=25, pady=25)

        # Strategy Selection with Checkboxes
        self.create_section_header(inner, "Trading Strategies (Select Multiple)")
        
        strategy_container = ctk.CTkFrame(inner, fg_color="#21262d", corner_radius=8)
        strategy_container.pack(fill='x', pady=(0, 20))
        
        strategy_inner = ctk.CTkFrame(strategy_container, fg_color="transparent")
        strategy_inner.pack(fill='both', padx=12, pady=12)

        strategy_display_names = {
            'vwap_crossover_strategy': 'VWAP Crossover',
            'opening_range_strategy': 'Opening Range Breakout',
            'bollinger_band_reversion_strategy': 'Bollinger Band Mean Reversion'
        }

        for strategy_key, strategy_var in self.selected_strategies.items():
            row = ctk.CTkFrame(strategy_inner, fg_color="transparent")
            row.pack(fill='x', pady=4)
            
            checkbox = ctk.CTkCheckBox(
                row,
                text=strategy_display_names[strategy_key],
                variable=strategy_var,
                font=ctk.CTkFont(family="Helvetica", size=11),
                text_color="#e6edf3",
                fg_color="#238636",
                hover_color="#2ea043",
                checkbox_width=20,
                checkbox_height=20
            )
            checkbox.pack(side='left', anchor='w')
            
            edit_btn = ctk.CTkButton(
                row,
                text="✏️ Edit",
                width=70,
                height=28,
                font=ctk.CTkFont(family="Helvetica", size=10),
                fg_color="#30363d",
                hover_color="#484f58",
                text_color="#8b949e",
                corner_radius=6,
                command=lambda sk=strategy_key: self.edit_strategy(sk)
            )
            edit_btn.pack(side='right')

        # Select/Deselect All
        btn_row = ctk.CTkFrame(strategy_container, fg_color="transparent")
        btn_row.pack(fill='x', padx=12, pady=(0, 10))
        
        ctk.CTkButton(
            btn_row,
            text="✓ Select All",
            width=100,
            height=28,
            font=ctk.CTkFont(family="Helvetica", size=10),
            fg_color="#30363d",
            hover_color="#484f58",
            text_color="#8b949e",
            corner_radius=6,
            command=self.select_all_strategies
        ).pack(side='left', padx=(0, 5))
        
        ctk.CTkButton(
            btn_row,
            text="✗ Deselect All",
            width=100,
            height=28,
            font=ctk.CTkFont(family="Helvetica", size=10),
            fg_color="#30363d",
            hover_color="#484f58",
            text_color="#8b949e",
            corner_radius=6,
            command=self.deselect_all_strategies
        ).pack(side='left')

        # Risk Management
        self.create_section_header(inner, "Risk Management")
        self.create_sleek_input(inner, "Stop Loss (pips)", self.sl_pips)
        self.create_sleek_input(inner, "Take Profit (pips)", self.tp_pips)

        # Execution Costs
        self.create_section_header(inner, "Execution Costs")
        self.create_sleek_input(inner, "Spread (pips)", self.spread_pips)
        self.create_sleek_input(inner, "Slippage (pips)", self.slippage_pips)

    def edit_strategy(self, strategy_key):
        """Open strategy editor - needs StrategyEditorWindow from part 1"""
        messagebox.showinfo("Edit Strategy", f"Strategy editor for {strategy_key}\n\nIntegrate StrategyEditorWindow from part 1")

    def select_all_strategies(self):
        for var in self.selected_strategies.values():
            var.set(True)

    def deselect_all_strategies(self):
        for var in self.selected_strategies.values():
            var.set(False)

    # ══════════════════════════════════════════════════════════════════
    # ACCOUNT SECTION
    # ══════════════════════════════════════════════════════════════════
    def create_account_section(self):
        section = ctk.CTkFrame(self.content_frame, fg_color="transparent")
        self.sections['account'] = section

        ctk.CTkLabel(
            section,
            text="Account Settings",
            font=ctk.CTkFont(family="Helvetica", size=16, weight="normal"),
            text_color="#e6edf3",
            anchor="w"
        ).pack(anchor='w', pady=(0, 20))

        content = ctk.CTkFrame(section, fg_color="#000000", corner_radius=10)
        content.pack(fill='both', expand=True)

        inner = ctk.CTkFrame(content, fg_color="transparent")
        inner.pack(fill='both', expand=True, padx=25, pady=25)

        self.create_section_header(inner, "Capital & Leverage")
        self.create_sleek_input(inner, "Initial Balance ($)", self.initial_balance)
        self.create_sleek_input(inner, "Leverage", self.leverage, is_combobox=True,
                               values=["1", "10", "20", "30", "50", "100", "200", "500"])

        self.create_section_header(inner, "Position Sizing")
        self.create_sleek_input(inner, "Risk % per Trade", self.risk_percent)

    # ══════════════════════════════════════════════════════════════════
    # RESULTS SECTION - COMPLETE WITH CHARTS AND METRICS
    # ══════════════════════════════════════════════════════════════════
    def create_results_section(self):
        """COMPLETE results section with full visualization"""
        section = ctk.CTkFrame(self.content_frame, fg_color="transparent")
        self.sections['results'] = section

        # Main scrollable container
        self.results_scroll = ctk.CTkScrollableFrame(
            section,
            fg_color="transparent",
            scrollbar_button_color="#30363d",
            scrollbar_button_hover_color="#484f58"
        )
        self.results_scroll.pack(fill='both', expand=True)

        # Header
        ctk.CTkLabel(
            self.results_scroll,
            text="Multi-Strategy Results",
            font=ctk.CTkFont(family="Helvetica", size=16, weight="normal"),
            text_color="#e6edf3",
            anchor="w"
        ).pack(anchor='w', pady=(0, 15))

        # Results container (populated dynamically)
        self.results_container = ctk.CTkFrame(self.results_scroll, fg_color="transparent")
        self.results_container.pack(fill='both', expand=True)

    def display_multi_strategy_results(self, results):
        """Display complete results with charts and metrics for each strategy"""
        # Clear container
        for widget in self.results_container.winfo_children():
            widget.destroy()
        
        # Store results
        self.all_results = {}
        for result in results:
            if result['success']:
                self.all_results[result['strategy_name']] = result
        
        if not self.all_results:
            ctk.CTkLabel(
                self.results_container,
                text="No successful strategy results",
                font=ctk.CTkFont(family="Helvetica", size=14),
                text_color="#f85149"
            ).pack(pady=50)
            return
        
        # Display each strategy with FULL visualization
        for idx, (strategy_name, result) in enumerate(self.all_results.items()):
            self.create_complete_strategy_result(strategy_name, result, idx)
        
        self.report_button.configure(state="normal")
        self.show_section('results')

    def create_complete_strategy_result(self, strategy_name, result, index):
        """Create COMPLETE result display with charts and metrics"""
        display_names = {
            'vwap_crossover_strategy': 'VWAP Crossover',
            'opening_range_strategy': 'Opening Range Breakout',
            'bollinger_band_reversion_strategy': 'Bollinger Band Mean Reversion'
        }
        
        display_name = display_names.get(strategy_name, strategy_name)
        metrics = result['metrics']
        trades_df = result['trades_df']
        
        # Main strategy card
        card = ctk.CTkFrame(self.results_container, fg_color="#0d1117", corner_radius=10)
        card.pack(fill='both', expand=True, pady=(0 if index == 0 else 20, 0))
        
        # ═══════════════════════════════════════════════════════════════
        # HEADER
        # ═══════════════════════════════════════════════════════════════
        header = ctk.CTkFrame(card, fg_color="#161b22", corner_radius=8)
        header.pack(fill='x', padx=15, pady=15)
        
        ctk.CTkLabel(
            header,
            text=f"📊 {display_name}",
            font=ctk.CTkFont(family="Helvetica", size=14, weight="bold"),
            text_color="#e6edf3",
            anchor="w"
        ).pack(side='left', padx=12, pady=10)
        
        # ═══════════════════════════════════════════════════════════════
        # SUMMARY METRICS (6 KEY METRICS)
        # ═══════════════════════════════════════════════════════════════
        if not trades_df.empty:
            summary_frame = ctk.CTkFrame(card, fg_color="transparent")
            summary_frame.pack(fill='x', padx=15, pady=(0, 15))
            
            for i in range(6):
                summary_frame.columnconfigure(i, weight=1)
            
            total_trades = len(trades_df)
            win_rate = metrics.get('win_rate_%', 0)
            total_pnl = metrics.get('total_pnl_$', 0)
            total_pips = metrics.get('total_pips', 0)
            total_return = metrics.get('total_return_%', 0)
            final_balance = metrics.get('final_balance_$', self.initial_balance.get())
            
            summary_data = [
                ('Trades', str(total_trades), '#e6edf3'),
                ('Win Rate', f"{win_rate:.1f}%", '#3fb950' if win_rate >= 50 else '#f85149'),
                ('P&L', f"${total_pnl:,.0f}", '#3fb950' if total_pnl >= 0 else '#f85149'),
                ('Pips', f"{total_pips:+.0f}", '#3fb950' if total_pips >= 0 else '#f85149'),
                ('Returns', f"{total_return:+.2f}%", '#3fb950' if total_return >= 0 else '#f85149'),
                ('Final Balance', f"${final_balance:,.0f}", '#3fb950' if final_balance > self.initial_balance.get() else '#f85149')
            ]
            
            for col, (label, value, color) in enumerate(summary_data):
                metric_card = ctk.CTkFrame(summary_frame, fg_color="#161b22", corner_radius=6)
                metric_card.grid(row=0, column=col, sticky='nsew', padx=4, pady=4)
                
                inner = ctk.CTkFrame(metric_card, fg_color="transparent")
                inner.pack(fill='both', expand=True, padx=8, pady=6)
                
                ctk.CTkLabel(
                    inner,
                    text=label,
                    font=ctk.CTkFont(family="Helvetica", size=9),
                    text_color="#6e7681",
                    anchor="w"
                ).pack(anchor='w')
                
                ctk.CTkLabel(
                    inner,
                    text=value,
                    font=ctk.CTkFont(family="Helvetica", size=13, weight="bold"),
                    text_color=color,
                    anchor="w"
                ).pack(anchor='w', pady=(2, 0))
        
        # ═══════════════════════════════════════════════════════════════
        # MATPLOTLIB CHARTS (2x2 GRID)
        # ═══════════════════════════════════════════════════════════════
        if not trades_df.empty:
            charts_label = ctk.CTkLabel(
                card,
                text="PERFORMANCE CHARTS",
                font=ctk.CTkFont(family="Helvetica", size=10, weight="bold"),
                text_color="#6e7681",
                anchor="w"
            )
            charts_label.pack(anchor='w', padx=15, pady=(10, 8))
            
            charts_container = ctk.CTkFrame(card, fg_color="#0d1117", corner_radius=8)
            charts_container.pack(fill='both', expand=True, padx=15, pady=(0, 15))
            
            # Create matplotlib figure
            fig = self.create_strategy_charts(trades_df, metrics)
            
            # Embed in tkinter
            canvas = FigureCanvasTkAgg(fig, master=charts_container)
            canvas.draw()
            canvas.get_tk_widget().pack(fill='both', expand=True, padx=10, pady=10)
        
        # ═══════════════════════════════════════════════════════════════
        # DETAILED METRICS TABLE
        # ═══════════════════════════════════════════════════════════════
        if metrics:
            metrics_label = ctk.CTkLabel(
                card,
                text="DETAILED METRICS",
                font=ctk.CTkFont(family="Helvetica", size=10, weight="bold"),
                text_color="#6e7681",
                anchor="w"
            )
            metrics_label.pack(anchor='w', padx=15, pady=(10, 8))
            
            metrics_container = ctk.CTkFrame(card, fg_color="#0d1117", corner_radius=8)
            metrics_container.pack(fill='x', padx=15, pady=(0, 15))
            
            metrics_grid = ctk.CTkFrame(metrics_container, fg_color="transparent")
            metrics_grid.pack(fill='x', padx=15, pady=15)
            
            # Configure 5 columns
            for i in range(5):
                metrics_grid.columnconfigure(i, weight=1)
            
            # Display all metrics in grid
            metrics_list = list(metrics.items())
            
            for idx, (key, val) in enumerate(metrics_list):
                row = idx // 5
                col = idx % 5
                self.create_metric_card(metrics_grid, key, val, row, col)

    def create_strategy_charts(self, trades_df, metrics):
        """Create 2x2 matplotlib chart grid for a strategy"""
        plt.style.use('dark_background')
        
        BG_COLOR = '#0d1117'
        GRID_COLOR = '#1a1a1a'
        TEXT_COLOR = '#888888'
        ACCENT_GREEN = '#3fb950'
        ACCENT_RED = '#f85149'
        
        # Create figure with responsive sizing
        window_width = self.master.winfo_width()
        chart_width = max(12, min(16, (window_width - 400) / 100))
        chart_height = max(6, min(8, chart_width * 0.5))
        
        fig = Figure(figsize=(chart_width, chart_height), facecolor=BG_COLOR, edgecolor=BG_COLOR)
        fig.subplots_adjust(left=0.06, right=0.98, top=0.92, bottom=0.10, hspace=0.40, wspace=0.22)
        
        # ─── TOP LEFT: Cumulative Returns ───
        ax1 = fig.add_subplot(2, 2, 1, facecolor=BG_COLOR)
        
        trades_df_sorted = trades_df.sort_values('exit_time')
        cumulative_pnl = trades_df_sorted['monetary_pnl'].cumsum()
        cumulative_returns = (cumulative_pnl / self.initial_balance.get()) * 100
        
        ax1.fill_between(trades_df_sorted['exit_time'], cumulative_returns, 0,
                        where=(cumulative_returns >= 0), color=ACCENT_GREEN, alpha=0.3)
        ax1.fill_between(trades_df_sorted['exit_time'], cumulative_returns, 0,
                        where=(cumulative_returns < 0), color=ACCENT_RED, alpha=0.3)
        ax1.plot(trades_df_sorted['exit_time'], cumulative_returns,
                color=ACCENT_GREEN, linewidth=1.5)
        ax1.axhline(y=0, color='#30363d', linestyle='-', linewidth=0.5)
        
        ax1.set_title('Cumulative Returns', fontsize=10, fontweight='bold',
                     color='#e6edf3', pad=6, fontfamily='Helvetica')
        ax1.tick_params(colors=TEXT_COLOR, labelsize=7)
        ax1.set_ylabel('Return (%)', fontsize=8, color=TEXT_COLOR, fontfamily='Helvetica')
        ax1.grid(False)
        for spine in ax1.spines.values():
            spine.set_color('#21262d')
        
        # ─── TOP RIGHT: P&L Distribution ───
        ax2 = fig.add_subplot(2, 2, 2, facecolor=BG_COLOR)
        
        returns = trades_df['monetary_pnl']
        n, bins, patches = ax2.hist(returns, bins=25, color=ACCENT_GREEN,
                                   alpha=0.8, edgecolor=BG_COLOR, linewidth=0.5)
        
        for i, patch in enumerate(patches):
            if bins[i] < 0:
                patch.set_facecolor(ACCENT_RED)
            else:
                patch.set_facecolor(ACCENT_GREEN)
        
        mean_return = returns.mean()
        ax2.axvline(x=mean_return, color='#e6edf3', linestyle='--',
                   linewidth=1, label=f'Mean: ${mean_return:.2f}')
        ax2.axvline(x=0, color='#30363d', linestyle='-', linewidth=0.5)
        ax2.legend(loc='upper right', fontsize=7, facecolor=BG_COLOR,
                  edgecolor='#21262d', labelcolor='#8b949e')
        
        ax2.set_title('Trade P&L Distribution', fontsize=10, fontweight='bold',
                     color='#e6edf3', pad=6, fontfamily='Helvetica')
        ax2.tick_params(colors=TEXT_COLOR, labelsize=7)
        ax2.set_xlabel('P&L ($)', fontsize=8, color=TEXT_COLOR, fontfamily='Helvetica')
        ax2.set_ylabel('Frequency', fontsize=8, color=TEXT_COLOR, fontfamily='Helvetica')
        ax2.grid(False)
        for spine in ax2.spines.values():
            spine.set_color('#21262d')
        
        # ─── BOTTOM LEFT: Drawdown ───
        ax3 = fig.add_subplot(2, 2, 3, facecolor=BG_COLOR)
        
        cumulative_pnl = trades_df_sorted['monetary_pnl'].cumsum()
        equity_curve = self.initial_balance.get() + cumulative_pnl
        running_max = equity_curve.expanding().max()
        drawdown = ((equity_curve - running_max) / running_max) * 100
        
        ax3.fill_between(trades_df_sorted['exit_time'], drawdown, 0,
                        color=ACCENT_RED, alpha=0.4)
        ax3.plot(trades_df_sorted['exit_time'], drawdown,
                color=ACCENT_RED, linewidth=1.5)
        
        ax3.set_title('Drawdown', fontsize=10, fontweight='bold',
                     color='#e6edf3', pad=6, fontfamily='Helvetica')
        ax3.tick_params(colors=TEXT_COLOR, labelsize=7)
        ax3.set_ylabel('Drawdown (%)', fontsize=8, color=TEXT_COLOR, fontfamily='Helvetica')
        ax3.grid(False)
        for spine in ax3.spines.values():
            spine.set_color('#21262d')
        
        # ─── BOTTOM RIGHT: Win/Loss Pie ───
        ax4 = fig.add_subplot(2, 2, 4, facecolor=BG_COLOR)
        
        wins = (trades_df['monetary_pnl'] > 0).sum()
        losses = (trades_df['monetary_pnl'] <= 0).sum()
        
        colors = [ACCENT_GREEN, ACCENT_RED]
        explode = (0.02, 0.02)
        
        wedges, texts, autotexts = ax4.pie(
            [wins, losses],
            labels=['Wins', 'Losses'],
            autopct='%1.1f%%',
            colors=colors,
            explode=explode,
            startangle=90,
            textprops={'color': '#e6edf3', 'fontsize': 9, 'fontfamily': 'Helvetica'}
        )
        
        for autotext in autotexts:
            autotext.set_color('#ffffff')
            autotext.set_fontweight('bold')
            autotext.set_fontsize(10)
        
        ax4.set_title('Win/Loss Ratio', fontsize=10, fontweight='bold',
                     color='#e6edf3', pad=6, fontfamily='Helvetica')
        
        return fig

    def create_metric_card(self, parent, key, value, row, col):
        """Create individual metric card with formatting"""
        card = ctk.CTkFrame(parent, fg_color="#161b22", corner_radius=6)
        card.grid(row=row, column=col, sticky='nsew', padx=4, pady=4)
        
        inner = ctk.CTkFrame(card, fg_color="transparent")
        inner.pack(fill='both', expand=True, padx=8, pady=6)
        
        name = key.replace('_', ' ').title()
        ctk.CTkLabel(
            inner,
            text=name,
            font=ctk.CTkFont(family="Helvetica", size=8),
            text_color="#6e7681",
            anchor="w"
        ).pack(anchor='w')
        
        # Format value with color
        if isinstance(value, float):
            if 'balance' in key.lower() or 'pnl' in key.lower():
                value_str = f"${value:,.2f}"
                color = "#3fb950" if value >= 0 else "#f85149"
            elif 'rate' in key.lower() or '%' in key:
                value_str = f"{value:.2f}%"
                color = "#3fb950" if value >= 50 else "#f85149"
            elif 'ratio' in key.lower() or 'factor' in key.lower():
                value_str = f"{value:.2f}"
                color = "#3fb950" if value > 1 else "#f85149"
            elif 'drawdown' in key.lower():
                value_str = f"{value:.2f}%"
                color = "#f85149"
            else:
                value_str = f"{value:.2f}"
                color = "#e6edf3"
        elif isinstance(value, int):
            value_str = f"{value:,}"
            color = "#e6edf3"
        else:
            value_str = str(value)
            color = "#e6edf3"
        
        ctk.CTkLabel(
            inner,
            text=value_str,
            font=ctk.CTkFont(family="Helvetica", size=11, weight="bold"),
            text_color=color,
            anchor="w"
        ).pack(anchor='w', pady=(2, 0))

    # ══════════════════════════════════════════════════════════════════
    # HELPER METHODS
    # ══════════════════════════════════════════════════════════════════
    def create_section_header(self, parent, text):
        ctk.CTkLabel(
            parent,
            text=text,
            font=ctk.CTkFont(family="Helvetica", size=12, weight="bold"),
            text_color="#8b949e",
            anchor="w"
        ).pack(anchor='w', pady=(12, 6))

    def create_sleek_input(self, parent, label_text, variable, is_combobox=False, values=None):
        frame = ctk.CTkFrame(parent, fg_color="transparent")
        frame.pack(fill='x', pady=5)

        ctk.CTkLabel(
            frame,
            text=label_text,
            font=ctk.CTkFont(family="Helvetica", size=11),
            text_color="#8b949e",
            width=120,
            anchor="w"
        ).pack(side='left')

        if is_combobox:
            widget = ctk.CTkComboBox(
                frame,
                variable=variable,
                values=values or [],
                font=ctk.CTkFont(family="Helvetica", size=11),
                fg_color="#21262d",
                border_color="#30363d",
                button_color="#30363d",
                button_hover_color="#484f58",
                text_color="#e6edf3",
                dropdown_fg_color="#000000",
                dropdown_hover_color="#21262d",
                dropdown_text_color="#e6edf3",
                corner_radius=6,
                height=32,
                width=240
            )
        else:
            widget = ctk.CTkEntry(
                frame,
                textvariable=variable,
                font=ctk.CTkFont(family="Helvetica", size=11),
                fg_color="#21262d",
                border_color="#30363d",
                text_color="#e6edf3",
                corner_radius=6,
                height=32,
                width=240
            )

        widget.pack(side='left', padx=(12, 0))
        return widget

    def update_status(self, text, color="#8b949e"):
        self.status_text.set(text)
        self.status_label.configure(text_color=color)

    def get_desktop_path(self):
        desktop_path = os.path.join(os.path.expanduser("~"), "Desktop")
        if os.path.exists(desktop_path):
            return desktop_path
        return os.path.expanduser("~")

    def select_data_folder(self):
        new_folder = filedialog.askdirectory(title="Select Main Data Folder")
        if new_folder:
            self.data_folder = pathlib.Path(new_folder)
            folder_text = str(self.data_folder)
            if len(folder_text) > 45:
                folder_text = "..." + folder_text[-42:]
            self.folder_label.configure(text=folder_text)
            self.update_status(f"Data folder updated", "#238636")

    # ══════════════════════════════════════════════════════════════════
    # BACKTEST EXECUTION (PLACEHOLDER - INTEGRATE FROM PART 1)
    # ══════════════════════════════════════════════════════════════════
    def start_backtest_thread(self):
        """Start multi-strategy backtest - integrate with part 1"""
        selected = [name for name, var in self.selected_strategies.items() if var.get()]
        
        if not selected:
            messagebox.showwarning("No Strategy", "Please select at least one strategy")
            return
        
        messagebox.showinfo("Demo Mode", 
                           f"Selected {len(selected)} strategies.\n\n" +
                           "Integrate run_parallel_backtests from part 1 here.\n\n" +
                           "This will:\n" +
                           "1. Load data once\n" +
                           "2. Run strategies in parallel\n" +
                           "3. Display results with charts")
        
        # Demo: Create fake results for visualization
        self.create_demo_results()

    def create_demo_results(self):
        """Create demo results for testing visualization"""
        demo_results = []
        
        for strategy_name, var in self.selected_strategies.items():
            if not var.get():
                continue
            
            # Create demo trades
            dates = pd.date_range(start='2024-01-01', end='2024-12-31', freq='D')
            n_trades = 50
            
            trades_df = pd.DataFrame({
                'trade_number': [f"{i:05d}" for i in range(1, n_trades+1)],
                'entry_time': pd.Series(dates[:n_trades]),
                'exit_time': pd.Series(dates[1:n_trades+1]),
                'signal': ['BUY' if i % 2 == 0 else 'SELL' for i in range(n_trades)],
                'monetary_pnl': np.random.randn(n_trades) * 100 + 20,
                'pips_pnl': np.random.randn(n_trades) * 10 + 2,
            })
            
            # Calculate metrics
            win_rate = (trades_df['monetary_pnl'] > 0).mean() * 100
            total_pnl = trades_df['monetary_pnl'].sum()
            total_pips = trades_df['pips_pnl'].sum()
            
            metrics = {
                'total_trades': n_trades,
                'win_rate_%': win_rate,
                'total_pnl_$': total_pnl,
                'total_pips': total_pips,
                'total_return_%': (total_pnl / self.initial_balance.get()) * 100,
                'final_balance_$': self.initial_balance.get() + total_pnl,
                'sharpe_ratio': np.random.uniform(0.5, 2.5),
                'max_drawdown_%': np.random.uniform(-5, -20),
                'profit_factor': np.random.uniform(1.2, 2.5),
            }
            
            demo_results.append({
                'strategy_name': strategy_name,
                'trades_df': trades_df,
                'metrics': metrics,
                'success': True,
                'error': None
            })
        
        self.display_multi_strategy_results(demo_results)

    def generate_all_reports(self):
        """Generate reports for all strategies"""
        if not self.all_results:
            messagebox.showwarning("No Data", "No results available")
            return
        
        messagebox.showinfo("Reports", f"Would generate {len(self.all_results)} HTML reports")

    def export_all_to_csv(self):
        """Export all results to CSV"""
        if not self.all_results:
            messagebox.showwarning("No Data", "No results available")
            return
        
        messagebox.showinfo("Export", f"Would export {len(self.all_results)} CSV files")

    def clear_cache(self):
        """Clear data cache"""
        messagebox.showinfo("Cache", "Cache cleared")
        self.update_status("Cache cleared", "#3fb950")


# ══════════════════════════════════════════════════════════════════════
# MAIN
# ══════════════════════════════════════════════════════════════════════
if __name__ == '__main__':
    if os.name == 'nt':
        mp.freeze_support()
    
    app = ctk.CTk()
    backtester = BacktesterUI(app)
    app.mainloop()

findfont: Font family 'Helvetica' not found.
findfont: Font family 'Helvetica' not found.
findfont: Font family 'Helvetica' not found.
findfont: Font family 'Helvetica' not found.
findfont: Font family 'Helvetica' not found.
findfont: Font family 'Helvetica' not found.
findfont: Font family 'Helvetica' not found.
findfont: Font family 'Helvetica' not found.
findfont: Font family 'Helvetica' not found.
findfont: Font family 'Helvetica' not found.
findfont: Font family 'Helvetica' not found.
findfont: Font family 'Helvetica' not found.
findfont: Font family 'Helvetica' not found.
findfont: Font family 'Helvetica' not found.
findfont: Font family 'Helvetica' not found.
findfont: Font family 'Helvetica' not found.
findfont: Font family 'Helvetica' not found.
findfont: Font family 'Helvetica' not found.
findfont: Font family 'Helvetica' not found.
findfont: Font family 'Helvetica' not found.
findfont: Font family 'Helvetica' not found.
findfont: Font family 'Helvetica' not found.
findfont: 